In [1]:
%run "./ADP_Farmatic_Def"

In [2]:
%run "./ADP_Spain_MDM_Def"

In [3]:
%run "../Libraries/ADP_QA"

In [4]:
#################################################################################
""" Farmatic Canonize Files Library

"""
 #Who                 When           What
 #Victor Salesa       18/12/2018     Initial VersionRefresh GenerateCanonicalFarmaticSelloutcanonical table:CTL_PROCESS_FILE_QA
 #Victor Salesa       19/12/2018     Added TrimiGenerateCanonicalFarmaticSelloutng to all fields
 #Victor Salesa       04/04/2019     Added RecoverTemporaryFarmaticSellout method
 #Victor Salesa       12/04/2019     cHANGED __wholeFile__ TO INCLUDE ENCODING PARAMETER
 #Victor Salesa       21/04/2019     Added Variables for paths and table names
 #Victor Salesa       29/04/2019     FieldFormatValidationsFarmaticSellout:Included new QA fields (NUM_VALIDATED_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE)
 #Victor Salesa       29/04/2019     GenerateTemporaryFarmaticSellout:Included new QA fields (NUM_ERROR_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE) 
 #Victor Salesa       29/04/2019     GenerateTemporaryFarmaticSellout:Renamed JOB_ID field with NUM_ROWS_TOTAL
 #Victor Salesa       29/04/2019     GenerateTemporaryFarmaticSellout:Drop Cols ROW_KO_PMS,ROW_KO_MDM,NUMBER_OF_VALIDATED_FIELDS_PMS,NUMBER_OF_VALIDATED_FIELDS_MDM,"OLD" ROW_OK, ROW_KO
 #Victor Salesa       29/04/2019     GenerateQAFarmaticSellout:Included new QA fields (NUM_VALIDATED_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE)
 #Victor Salesa       29/04/2019     GenerateQAFarmaticSellout:Included new QA fields (NUM_ERROR_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE) 
################################################################################sa

__DELETE_FILES_ENABLED__ = True

####CTL JOB DEF#########################################################################################################################

##Pending CTL.JOB table to be corrected to have JOB_ID as an integer with 31 digits
__CTL_JOB__DB_TABLE_NAME__ = "CTL.JOB_"

####FARMATIC SELLOUT DEF################################################################################################################

__PHARMATIC_CANONICAL_STG_SELLOUT_TMP_H_TABLE_NAME__ = 'TMP_STG_T_SELL_OUT'
__PHARMATIC_CANONICAL_STG_SELLOUT_TMP_FILE_PATH__  = __PHARMATIC_CANONICAL_BASE_PATH__+'SL/'+ 'TMP_STG_T_SELL_OUT'


__PHARMATIC_CANONICAL_STG_SELLOUT_H_TABLE_NAME__ = 'ADP_STG_T_SELL_OUT' 
__PHARMATIC_CANONICAL_STG_SELLOUT_FILE_PATH__  = __PHARMATIC_CANONICAL_BASE_PATH__+'SL/'+ 'STG_T_SELL_OUT'

__PHARMATIC_CANONICAL_STG_SELLOUT_DB_TABLE_NAME__ ='ADP.STG_T_SELL_OUT'

####QUALITY DEF##########################################################################################################################

__QUALITY_PFV_H_TABLE_NAME__ = 'CTL_PROCESS_FILE_VAL'
__QUALITY_PFV_H_FILE_PATH__  = __QUALITY_BASE_PATH__ + 'ctl_process_file_val'

__QUALITY_PFV_DB_TABLE_NAME__='CTL.STG_PROCESS_FILE_VAL' 

__QUALITY_PFE_H_TABLE_NAME__ = 'CTL_PROCESS_FILE_ERROR'
__QUALITY_PFE_H_FILE_PATH__  = __QUALITY_BASE_PATH__ + 'ctl_process_file_error'

__QUALITY_PFE_DB_TABLE_NAME__='CTL.STG_PROCESS_FILE_ERROR' 

__QUALITY_PFQ_H_TABLE_NAME__ = 'CTL_PROCESS_FILE_QA'
__QUALITY_PFQ_H_FILE_PATH__ = __QUALITY_BASE_PATH__+'ctl_process_file_qa' + '/table'

__QUALITY_PFQ_DB_TABLE_NAME__ = 'CTL.STG_PROCESS_FILE_QA'

########################################################################################################################################

def __wholeFile__(wholeTextFilesLine,force_decode='ISO-8859-1'):
  """Converts 1 single line tuple returned from the wholeTextFilesfunction to a DataframeRow.

    Parameters:
      wholeTextFilesLine   -- 1 single line tuple returned from wholeTextFiles function with (filepath,filecontents)
      force_decode         -- decode chasrset

    Return:
      pyspark.sql.Row      -- Dataframe Row with the following fields name,line_id,value
                           -- FILE_PATH: name of the file
                           -- FILE_LINE_NUM: line index inside the file
                           -- FILE_LINE_CONTENT:    content of each single line of the file
  """
  #Who                 When           What
  #Victor Salesa       18/12/2018     Initial version
  #Victor Salesa       28/01/2019     Added .decode('ISO-8859-1') to decode latin1 files
  #Ana Perez           27/03/2019     Included log managment and exception managment
  #Victor Salesa       11/04/2019     Added force_decode parameter to be able to change de decoding used
  #                                   Included chardet library to detect encoding in case the encoding is not defined (slower)
  try:
    #Get name of the file
    name  = wholeTextFilesLine[0]
    #Get the content of the file and generate a list with the lines splitted

    #Take the text byte array
    text_data_array = wholeTextFilesLine[1]

    #If the encoding is left as blank it will force lookup the encoding so will take more time to decode
    if force_decode=='':
      charset =    chardet.detect(text_data_array)['encoding']    
      force_decode = charset
    #end if force_decode=='':
    
    #Decode data based on force_decode parameter and then split by CRLF
    lines = text_data_array.decode(force_decode).split('\r\n')
    #Add line order to the list of lines splitted
    result_lines = [(i,s) for i, s in enumerate(lines)]
    result = [(name,result_line) for result_line in result_lines ]
    #Generate a Row with the name,line_id,linevalue
    row_result = [Row(FILE_PATH=name,FILE_LINE_NUM=result_line[0],FILE_LINE_CONTENT=result_line[1]) for result_line in result_lines]

    return row_result 
  except Exception as err:
#     ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
    raise Exception(err)
    
#############################################################################################################################  
  
def NormalizeFarmaticSellout(files,partitions=__PARTITIONS_DEFAULT__,debug=__DEBUG_DEFAULT__,sample=__SAMPLE_DEFAULT__,sample_quantity=__SAMPLE_QUANTITY_DEFAULT__):
  """Normalize a folder containing a list of files containing Farmatic Sellout information to a Big Dataframe with.

    Parameters:
      files                -- Path with wildcards of the files to be read
      partitions           -- Partitions to be used when using wholetextfiles
      debug                -- True for enable debug verbosing or False to not enable
      sample               -- To enable use a sample of the whole files retrieved to process them
      sample_quantity      -- In case sample is enabled sample_quantity to be tested

    Return:
      Dataframe            -- Dataframe containing the files with splitted data and filename line num 
  """
  #Who                 When           What
  #Victor Salesa       18/12/2018     Initial version
  #Victor Salesa       19/12/2018     Completed the code and Added new parameters  
  #Victor Salesa       08/01/2019     Added FILE_SPEC_VERSION,FILE_RELEASE_VERSION,FILE_ORIGIN,FILE_TYPE,COUNTRY to the Normalized output df
  #Victor Salesa       11/01/2019     Added PMS_CODE to Normalized Output 
  #Victor Salesa       14/01/2019     Added Validation type field to error output
  #Victor Salesa       15/01/2019     Added Delete old timestamps and error files
  #Victor Salesa       28/01/2019     Substituted wholeTextFiles with binaryFiles to solve Enconding issue.
  #Victor Salesa       30/01/2019     Added COUNTRY_CODE to CTL.PROCESS_FILE_VAL write
  #Ana Perez           11/02/2019     Replace delete_file function to blob_delete_file_sql
  #Ana Perez           11/02/2019     Replace yyyyMMddhhmmss pattern to yyyyMMddHHmmss 
  #Victor Salesa       21/02/2019     TZBIADP-201 ERROR CONTROL SYSTEM: VALIDATION FILE NAME PROCESS NormalizeFarmaticSellout:
  #                                       -Include delete control for last landing date deleting
  #Victor Salesa       27/02/2019     Change write to csv with write to parquet i
  #Victor Salesa       27/02/2019     Include GetDataFrameAsSchema to adapt schema to parquet file and database
  #Victor Salesa       18/03/2019     Changed START_DATE and END_DATE to be really start and end of the process when writing to CTL_PROCESS_FILE_VAL
  #Victor Salesa       21/03/2019     Added Code to Backup deleted files information when the process fails deleting some of the files
  #Ana Perez           29/03/2019     Included log managment and exception managment
  try:
    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)
    
    # Date and Time of the begining to this process
    start_date = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S') 
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Create wholetextfiles rdd", sys._getframe().f_code.co_name)
	
    textFile = sc.binaryFiles(files,partitions)

    ADP_log_debug(process, logger_name, level_action, log_level, "Persist readed files", sys._getframe().f_code.co_name)
	
    textFile.persist(StorageLevel.MEMORY_AND_DISK)
    textFile.count()
    
    #Take a sample of the files
    if sample==True:
      ADP_log_debug(process, logger_name, level_action, log_level, "Take a sample of files", sys._getframe().f_code.co_name) 
      textFile = sc.parallelize(textFile.take(sample_quantity),partitions)
      textFile.persist(StorageLevel.MEMORY_AND_DISK)
      textFile.count()
    
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Calculate line types to get Header and Detail Split", sys._getframe().f_code.co_name) 
	
    #Calculate line types to get Header and Detail Split
    textFileLineTypesDF = (textFile.flatMap(__wholeFile__).toDF()
                               .filter(col("FILE_PATH").substr(-20,2)=='LD')
                               .withColumn("RAW_NAME",split(col("FILE_PATH"),'/')[5])
                               .withColumn("FILE_NAME",concat(col("RAW_NAME").substr(lit(0),(length(col("RAW_NAME"))-21)),lit('.TXT') ))                            
                               .transform(SliceDFColumn("FILE_NAME",__SP_FILENAME_COLUMN_NAMES__,__SP_FILENAME_LENGHTS__))
                               .drop('pharmacy_unique_code')
                               .drop('data_date')
                               .withColumnRenamed('spec_version', 'FILE_SPEC_VERSION')
                               .withColumnRenamed('release_version', 'FILE_RELEASE_VERSION')
                               .withColumnRenamed('origin', 'FILE_ORIGIN')
                               .withColumnRenamed('file_type', 'FILE_TYPE')
                               .withColumn('COUNTRY',lit('ES'))     
                               .withColumn("LANDING_DATE",col("FILE_PATH").substr(-18,14))
                               .transform(SliceDFColumn("FILE_LINE_CONTENT",['RecordType'],[__RECORDTYPE_LEN__]))
                         )
    	
    ADP_log_debug(process, logger_name, level_action, log_level, "Persist sliced name files", sys._getframe().f_code.co_name)
    textFileLineTypesDF.persist(StorageLevel.MEMORY_AND_DISK)
    textFileLineTypesDF.count()
    
    #Create a window function to get the latest timestamp of the file
    windowSpec = Window.partitionBy(textFileLineTypesDF['FILE_NAME']).orderBy(textFileLineTypesDF['LANDING_DATE'].desc())

    #Create a window function to get the number of lines per file to get "empty file"
    windowEmptyFiles = Window.partitionBy(textFileLineTypesDF['FILE_NAME'],textFileLineTypesDF['LANDING_DATE']).orderBy(textFileLineTypesDF['FILE_NAME'].desc(),textFileLineTypesDF['LANDING_DATE'].desc())

    #Filter files to have just the latest timestamp version
    textFileLineTypesLastTimestampDF = (textFileLineTypesDF
                                      .withColumn("LATEST_LANDING_DATE",first("LANDING_DATE").over(windowSpec))
                                      .withColumn("FILE_LINES",count("FILE_LINE_CONTENT").over(windowEmptyFiles))
                                      .withColumn("FILE_NO_DATA",when(col("FILE_LINES")<=3,1).otherwise(0)  )                      
                                      .withColumn("LATEST_FILE_FLG",when(col("LANDING_DATE")==col("LATEST_LANDING_DATE"),1).otherwise(0))                                   
    )
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Persist Last timestamp files", sys._getframe().f_code.co_name)
    textFileLineTypesLastTimestampDF.persist(StorageLevel.MEMORY_AND_DISK)
    textFileLineTypesLastTimestampDF.count() 
  
    if __DELETE_FILES_ENABLED__ == True:
      ADP_log_debug(process, logger_name, level_action, log_level, "Old timestamp files to be deleted: " +\
               	    str(textFileLineTypesLastTimestampDF.filter(col("LATEST_FILE_FLG")==0).select("RAW_NAME").distinct().count()) + " files", sys._getframe().f_code.co_name)

      # Cache textFileLineTypesLastTimestampDF to avoid delete issues
      textFileLineTypesLastTimestampDF = textFileLineTypesLastTimestampDF.persist(StorageLevel.MEMORY_AND_DISK) 
        
      ## Delete old Timestamp files from folder as they are not going to be processed
      files_deleted = (textFileLineTypesLastTimestampDF.filter(col("LATEST_FILE_FLG")==0)
                                       .select("RAW_NAME")
                                       .distinct()
                                       .withColumn("deleted",blob_delete_file_sql(concat(lit(__PHARMATIC_TOBEPROCESSED_BASE_PATH__),col("RAW_NAME"))))
                                       .collect()
      )
      
      textFileLineTypesLastTimestampDF.count()
      
      if len(files_deleted)!=0:
        #Mount a dataframe with deleted files result
        result_delete_df = sc.parallelize(files_deleted).toDF()
        result_delete_df.cache()

        #Calculate Deleted Files
        total_deleted = (result_delete_df.agg(sum("deleted").alias("deleted_total")).collect())[0].deleted_total

        #Calculate Total Files
        total_rows = result_delete_df.count()

        #Generate Error if total_deleted < total_rows
        if total_deleted < total_rows:
          not_deleted = result_delete_df.filter(col("deleted")==0).collect()
          not_deleted_names = str([file.RAW_NAME for file in not_deleted]).replace("[","").replace("]","")
          raise Exception('Fail deleting bad files: '+ not_deleted_names)

      ADP_log_debug(process, logger_name, level_action, log_level, "Files Deleted: " + str(files_deleted), sys._getframe().f_code.co_name)
    
	#end if __DELETE_FILES_ENABLED__ == True
	
    ## Take just files with latest timestamp version
    textFileLineTypesLastTimestampDF = textFileLineTypesLastTimestampDF.filter(col("LATEST_FILE_FLG")==1)
  
    ADP_log_debug(process, logger_name, level_action, log_level, "Cache result to avoid recalculating RecordType" , sys._getframe().f_code.co_name)
   
    #Cache result to avoid recalculating RecordType
    textFileLineTypesLastTimestampDF = textFileLineTypesLastTimestampDF.cache()
    
  
    ##Add Process date to df
    ADP_log_debug(process, logger_name, level_action, log_level, "Add Process Date to Dataframe" , sys._getframe().f_code.co_name)
        
    processDate = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    textFileLineTypesLastTimestampDF = (textFileLineTypesLastTimestampDF
                         .withColumn("PROCESS_DATE", to_timestamp(lit(processDate), 'yyyy-MM-dd HH:mm:ss')) 
                         .withColumn("PROCESS_DATE",col("PROCESS_DATE").cast(StringType()))
                         .withColumn("PROCESS_DATE",date_format(col("PROCESS_DATE"),'yyyyMMddHHmmss'))
                        )
    ADP_log_debug(process, logger_name, level_action, log_level, "Calculate Header Split" , sys._getframe().f_code.co_name)
   
    #Calculate Header Split
    textFileHeaderDF   =  (textFileLineTypesLastTimestampDF.filter(col("RecordType") == "H")
                                .transform(SliceDFColumn("FILE_LINE_CONTENT",__SP_HEADER_COLUMN_NAMES__,__SP_HEADER_LENGHTS__))
                                .drop("FILE_LINE_CONTENT","RecordType")
                                .select("FILE_PATH","RAW_NAME","FILE_NAME","LANDING_DATE","PharmacyID","FileDate","ZipCode","ExternalPharmacyID",\
								        "FILE_LINES","FILE_NO_DATA","FILE_SPEC_VERSION","FILE_RELEASE_VERSION","FILE_ORIGIN","FILE_TYPE","COUNTRY","PROCESS_DATE")
                        )
    ADP_log_debug(process, logger_name, level_action, log_level, "Cache result to avoid recalculating Header Split" , sys._getframe().f_code.co_name)
     
    #Cache result to avoid recalculating Header Split
    textFileHeaderDF   = textFileHeaderDF.cache().alias('df_header')
	
    ADP_log_debug(process, logger_name, level_action, log_level, "Calculate Detail Split" , sys._getframe().f_code.co_name)
   
    #Calculate Detail Split
    textFileDetailDF   =  (textFileLineTypesLastTimestampDF.filter(col("RecordType") == "D")
                                 .transform(SliceDFColumn("FILE_LINE_CONTENT",__SP_DETAIL_COLUMN_NAMES__,__SP_DETAIL_LENGHTS__))
                                 .withColumn ("FILE_LEN_ROW_NOT_OK", when(length(col("FILE_LINE_CONTENT")) == lit(__SP_TOTAL_LENGHTS__),0).otherwise(1).cast(IntegerType())  )      
                                 .drop("FILE_PATH","RAW_NAME","FILE_LINE_CONTENT","RecordType","FILE_LINES","FILE_NO_DATA","FILE_SPEC_VERSION","FILE_RELEASE_VERSION","FILE_ORIGIN","FILE_TYPE","COUNTRY","PROCESS_DATE")   
                          )
    ADP_log_debug(process, logger_name, level_action, log_level, "Cache result to avoid recalculating Detail Split" , sys._getframe().f_code.co_name)
      
    #Cache result to avoid recalculating Detail Split
    textFileDetailDF   = textFileDetailDF.cache().alias('df_detail')
  
    ADP_log_debug(process, logger_name, level_action, log_level, "Define key columns to be used in the join" , sys._getframe().f_code.co_name)
      
    #Define key columns to be used in the join
    key_columns = ["FILE_NAME","LANDING_DATE"]

    ADP_log_debug(process, logger_name, level_action, log_level, "Define join condition for the join" , sys._getframe().f_code.co_name)
       
    #Define join condition for the join
    join_cond_generated = [ (col('df_header.'+field) == col('df_detail.'+field)) for field in key_columns]
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Exclude key columns of detail select clause" , sys._getframe().f_code.co_name)
    
    #Exclude key columns of detail select clause
    detail_selected_columns = [value for value in textFileDetailDF.columns if value not in key_columns]
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Generate Select List of columns" , sys._getframe().f_code.co_name)
    
    #Generate Select List of columns
    select_generated = [col("df_header.*")] + [col('df_detail.'+column) for column in detail_selected_columns]
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Join Header and Detail" , sys._getframe().f_code.co_name)
    
    #Join Header and Detail 
    textFileCompleteDF = (textFileHeaderDF.repartition(partitions)
                           .join(textFileDetailDF.repartition(partitions),join_cond_generated, how='left').select(*select_generated)
                         )
    
    #Create a window function to get the number of lines per file to get "empty file"
    windowFileErrors = Window.partitionBy(textFileCompleteDF['FILE_NAME'],textFileCompleteDF['LANDING_DATE'],textFileCompleteDF['PharmacyID']).orderBy(textFileLineTypesDF['FILE_NAME'].desc(),textFileLineTypesDF['LANDING_DATE'].desc(),textFileCompleteDF['PharmacyID'].desc())

    #Agregate lines for FILE_NO_DATA and FILE_LEN_ROW_NOT_OK 
    textFileCompletewithErrorsDF = (textFileCompleteDF.withColumn("FILE_NO_DATA",max("FILE_NO_DATA").over(windowFileErrors))
                                                   .withColumn("FILE_LEN_ROW_NOT_OK",sum(coalesce(col("FILE_LEN_ROW_NOT_OK"),lit(0)).cast(IntegerType())).over(windowFileErrors))
                                 )
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Aggregate Lines Not ok Start" , sys._getframe().f_code.co_name)
    
    textFileCompletewithErrorsDF.persist(StorageLevel.MEMORY_AND_DISK)
    textFileCompletewithErrorsDF.count()
    ADP_log_debug(process, logger_name, level_action, log_level, "Aggregate Lines Not ok End" , sys._getframe().f_code.co_name)
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Build Error DF to persist Start" , sys._getframe().f_code.co_name)	   
    
    textFileErrorsDF = (textFileCompletewithErrorsDF.select("FILE_NAME","LANDING_DATE","PharmacyID","FILE_NO_DATA","FILE_LEN_ROW_NOT_OK","RAW_NAME","FILE_PATH").distinct()
                         .withColumn("PHARMACY_CODE",col("PharmacyID"))
                         .drop("PharmacyID")
                         .withColumn("PMS_CODE",lit("FMT"))
                         .withColumn("COUNTRY_CODE",lit("ES"))
                         .withColumn("BUSINESS_AREA",lit("SL"))
                         .withColumn("LANDING_DATE",to_timestamp("LANDING_DATE", "yyyyMMddHHmmss"))
                         .withColumn("STATUS",lit(-2).cast(DecimalType(18,0)))
                         .withColumn("VALIDATION_TYPE",lit('I').cast(StringType()))
                         .withColumn("FILE_NO_DATA_TEXT",when(col("FILE_NO_DATA")==1,'"NO_DATA": "File does not have data"').otherwise('"NO_DATA": "OK"')  )
                         .withColumn("FILE_LEN_ROW_NOT_OK_TEXT",when(col("FILE_LEN_ROW_NOT_OK")==1,'"INVALID_STRUCTURE": "File does not have a valid structure"').otherwise('"INVALID_STRUCTURE": "OK"')  )
                         .withColumn("MESSAGE_TEXT"  ,  concat(lit("{"),col("FILE_NO_DATA_TEXT"),lit(","),col("FILE_LEN_ROW_NOT_OK_TEXT"),lit("}"))  )
                         .withColumn("ERROR_CODE"    ,  concat(lit("{"),lit('"NO_DATA":'),lit('"'),col("FILE_NO_DATA"),lit('"'),lit(","),lit('"INVALID_STRUCTURE":'),lit('"'),col("FILE_LEN_ROW_NOT_OK"),lit('"'),lit("}"))  )
                         .filter((col("FILE_NO_DATA")==1) | (col("FILE_LEN_ROW_NOT_OK")==1) )
                         .drop("FILE_NO_DATA","FILE_LEN_ROW_NOT_OK","FILE_NO_DATA_TEXT","FILE_LEN_ROW_NOT_OK_TEXT")
                       )
    
    # Cache textFileErrorsDF to avoid delete issues
    ADP_log_debug(process, logger_name, level_action, log_level, "Build Error DF to persist End" , sys._getframe().f_code.co_name)
    textFileErrorsDF = textFileErrorsDF.persist(StorageLevel.MEMORY_AND_DISK)
    textFileErrorsDF.count()
	
    if __DELETE_FILES_ENABLED__ == True:
      deleted = (textFileErrorsDF.select("RAW_NAME","FILE_PATH").distinct()
                       .withColumn("deleted",blob_delete_file_sql(concat(lit(__PHARMATIC_TOBEPROCESSED_BASE_PATH__),col("RAW_NAME"))))
                       .collect()
      )
      
      #Mount a dataframe with deleted files result
      if len(deleted)!=0:
        result_delete_df = sc.parallelize(deleted).toDF()
        result_delete_df.cache()

        #Calculate Deleted Files
        total_deleted = (result_delete_df.agg(sum("deleted").alias("deleted_total")).collect())[0].deleted_total

        #Calculate Total Files
        total_rows = result_delete_df.count()

        #Generate Error if total_deleted < total_rows
        if total_deleted < total_rows:
            not_deleted = result_delete_df.filter(col("deleted")==0).collect()
            not_deleted_names = str([file.RAW_NAME for file in not_deleted]).replace("[","").replace("]","")
            raise Exception('Fail deleting bad files: '+ not_deleted_names)
    
        ADP_log_debug(process, logger_name, level_action, log_level, "Deleted Files: " + str(deleted), sys._getframe().f_code.co_name)
        
	#end if __DELETE_FILES_ENABLED__ == True  
    
    ######################################################################################################################################################################## 
    #Drop Fields that will not go to disk and add Start and End date before writing to blob and db
    textFileErrorsDF = (textFileErrorsDF.drop("RAW_NAME","FILE_PATH")
                            .withColumn("START_DATE", to_timestamp(lit(start_date), 'yyyy-MM-dd HH:mm:ss'))
                            .withColumn("END_DATE",from_unixtime(unix_timestamp(current_timestamp())).cast(TimestampType()))
                       )

    ############################################################################################################################################################################
    textFileErrorsDF = GetDataFrameAsSchema(textFileErrorsDF,__CTL_PROCESS_FILE_VAL_SCHEMA__)
    
    #Write data to errors folder
    ADP_log_debug(process, logger_name, level_action, log_level, "Before saveAsCanonical CTL_PROCESS_FILE_VAL: " + str(textFileErrorsDF.count()) + " rows ", sys._getframe().f_code.co_name)
    saveAsCanonical(textFileErrorsDF,__QUALITY_PFV_H_FILE_PATH__,table_name=__QUALITY_PFV_H_TABLE_NAME__,mode='append',debug=debug)
    ADP_log_debug(process, logger_name, level_action, log_level, "After saveAsCanonical CTL_PROCESS_FILE_VAL ", sys._getframe().f_code.co_name)
    
  
    ##### Save to DB process_file_error QA data ###################################################################################################################################
    ADP_log_debug(process, logger_name, level_action, log_level, "Before saveToDB CTL_PROCESS_FILE_VAL: " + str(textFileErrorsDF.count()) + " rows ", sys._getframe().f_code.co_name)
    saveToDB(textFileErrorsDF,__QUALITY_PFV_DB_TABLE_NAME__,mode="append",debug=debug)
    ADP_log_debug(process, logger_name, level_action, log_level, "After saveToDB CTL_PROCESS_FILE_VAL", sys._getframe().f_code.co_name)
	
    #Filter files 
    ADP_log_debug(process, logger_name, level_action, log_level, "Remove Not Ok Lines Start", sys._getframe().f_code.co_name)
    textFileCompletewithErrorsDF = (textFileCompletewithErrorsDF
                                    .filter((col("FILE_NO_DATA")==0) & (col("FILE_LEN_ROW_NOT_OK")==0) )
                                    .withColumn("PMS_CODE",lit("FMT"))
                                    .drop("FILE_LEN_ROW_NOT_OK","FILE_NO_DATA","LATEST_LANDING_DATE")
                                   )
    
    textFileCompletewithErrorsDF.persist(StorageLevel.MEMORY_AND_DISK)
    textFileCompletewithErrorsDF.count()
    ADP_log_debug(process, logger_name, level_action, log_level, "Remove Not Ok Lines End", sys._getframe().f_code.co_name)
    
    #Trim spaces inside fields
    ADP_log_debug(process, logger_name, level_action, log_level, "Trim spaces inside fields", sys._getframe().f_code.co_name)
    textFileCompletewithErrorsDF = (reduce(
                lambda textFileCompletewithErrorsDF, col_name: textFileCompletewithErrorsDF.withColumn(col_name, trim(col(col_name))),
                textFileCompletewithErrorsDF.columns,
                textFileCompletewithErrorsDF)
            ).cache()
    ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name)
    
    return textFileCompletewithErrorsDF
  except Exception as err:
    if str(err).__contains__('Fail deleting bad files'):
      ADP_log_debug(process, logger_name, level_action, log_level, "In Exception clause: ", sys._getframe().f_code.co_name)
      not_deleted_names = [file.RAW_NAME for file in not_deleted]
      textFileErrorsDF = textFileErrorsDF.filter(textFileErrorsDF.RAW_NAME.isin(*not_deleted_names) == False)
      #Drop Fields that will not go to disk and add Start and End date before writing to blob and db
      textFileErrorsDF = (textFileErrorsDF.drop("RAW_NAME","FILE_PATH")
                              .withColumn("START_DATE", to_timestamp(lit(start_date), 'yyyy-MM-dd HH:mm:ss'))
                              .withColumn("END_DATE",from_unixtime(unix_timestamp(current_timestamp())).cast(TimestampType()))
                         )
      ############################################################################################################################################################################
      textFileErrorsDF = GetDataFrameAsSchema(textFileErrorsDF,__CTL_PROCESS_FILE_VAL_SCHEMA__)
      textFileErrorsDF.show(truncate=False)

      #Write data to errors folder
      ADP_log_debug(process, logger_name, level_action, log_level, "Before saveAsCanonical CTL_PROCESS_FILE_VAL: " + str(textFileErrorsDF.count()) + " rows ", sys._getframe().f_code.co_name)
      saveAsCanonical(textFileErrorsDF,__QUALITY_PFV_H_FILE_PATH__,table_name=__QUALITY_PFV_H_TABLE_NAME__,mode='append',debug=debug)
      ADP_log_debug(process, logger_name, level_action, log_level, "After saveAsCanonical CTL_PROCESS_FILE_VAL", sys._getframe().f_code.co_name)
      
      ##### Save to DB process_file_error QA data ###############################################################################################################################
      ADP_log_debug(process, logger_name, level_action, log_level, "Before saveToDB CTL_PROCESS_FILE_VAL: " + str(textFileErrorsDF.count()) + " rows ", sys._getframe().f_code.co_name)
      saveToDB(textFileErrorsDF,__QUALITY_PFV_DB_TABLE_NAME__,mode="append",debug=debug)
      ADP_log_debug(process, logger_name, level_action, log_level, "After saveToDB CTL_PROCESS_FILE_VAL", sys._getframe().f_code.co_name)
    #endif str(e).__contains__('Fail deleting bad files')
    
    ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
    raise Exception(err)
  
#############################################################################################################################

def FieldFormatValidationsFarmaticSellout(df,debug=__DEBUG_DEFAULT__,partitions=__PARTITIONS_DEFAULT__):
  """Validate Sellout Columns structure and Add a Column for each field with the validation Result.

    Parameters:
      df                   -- Dataframe with the result of the Normalize Process
      debug                -- True for enable debug verbosing or False to not enable
      partitions           -- Partitions defined

    Return:
      Dataframe            -- Dataframe containing the files with splitted data and the new validation columns and filename and line num 
  """
  #Who                 When           What
  #Victor Salesa       18/12/2018     Create Function Stub
  #Ana Perez           20/12/2018     Validations and Initial transformations
  #Ana Perez           11/01/2019     Calculates Negatives TotalOfReceipts
  #Ana Perez           25/01/2019     Includes new fields calculation
  #Ana Perez           31/01/2019     Fix Null values into PRODUCT_NET_WEIGTHED_PRICE field
  #Ana Perez           04/02/2019     Fix DATA_TYPE validations DecreasedValTotReceipt,DATA_EMPTY ProductCode 
  #Ana Perez           10/04/2019     Included log managment and exception managment
  #Ana Perez           15/04/2019     Included log debug managment
  #Ana Perez           22/04/2019     Included new QA files  (NUM_ERROR_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE)
  #Victor Salesa       29/04/2019     Included new QA fields (NUM_VALIDATED_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE)
  #Victor Salesa       29/04/2019     Drop Cols ROW_KO_PMS,ROW_KO_MDM,NUMBER_OF_VALIDATED_FIELDS_PMS,NUMBER_OF_VALIDATED_FIELDS_MDM,"OLD" ROW_OK, ROW_KO
  try:    
    
    ################################################################################################################################################
    #  VALIDATION FORMAT DATA FIELDS
    ################################################################################################################################################
    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)

    df_detail = (df
       .drop("RecordType")
       .withColumn("OPERATION_LINE_RAW", col("OperationLine"))
       .withColumn("OPERATION_DATE_RAW", col("OperationDate"))
       .withColumn("CUSTOMER_CODE_RAW", col("PharmacyID")) 
       .withColumn("NATIONAL_CODE_RAW", col("ProductCode"))
       .withColumn("PRODUCT_LINE_CODE_RAW", col("ProductCode")) 
       .withColumn("PRODUCT_NAME_RAW", col("ProductCode")) 
       .withColumn("CLASS_RAW", col("ProductCode"))
       .withColumn("CATEGORY_RAW", col("ProductCode")) 
       .withColumn("MANUFACTURER_CODE_RAW", col("ProductCode")) 
       .withColumn("MANUFACTURER_NAME_RAW", col("ProductCode")) 
       .withColumn("BRAND_RAW", col("ProductCode"))    
       .withColumn("TOT_RECEIPT_AMOUNT_RAW", col("TotalOfReceipt"))   
       .withColumn("TOT_RECEIPT_DISCOUNT_AMOUNT_RAW", col("DiscountValOvertTotReceipt"))   
       .withColumn("TOT_RECEIPT_DECREASE_AMOUNT_RAW", col("DecreasedValTotReceipt"))   
       .withColumn("PRODUCT_QTY_RAW", col("ProductPacks"))  
       .withColumn("PACK_SIZE_RAW", col("ProductPackSize")) 
       .withColumn("PRODUCT_PRICE_CATALOG_RAW", col("ProductPriceCatalog"))   
       .withColumn("PRODUCT_PRICE_RAW", col("ProductPrice"))
       .withColumn("DISCOUNT_VALUE_RAW", col("DiscountValue"))      
       .withColumn("PRODUCT_TOTAL_AMOUNT_RAW", col("ProductTotalPrice")) 
       .withColumn("REIMBURSEMENT_AMOUNT_RAW", col("ReimbursementValue"))  
       .withColumn("CONSUMER_AMOUNT_RAW", col("ConsumerValue"))    
       .withColumn("PAYMENT_MODE_RAW", col("PaymentMode"))       
       .withColumn("CONSUMER_GENDER_RAW", col("ConsumerGender")) 
       .withColumn("CONSUMER_TYPE_RAW", col("ConsumerType")) 
       .withColumn("OPERATION_TYPE_RAW", col("OperationIdent"))  
       .withColumn("PRESCRIPTION_MODE_RAW", col("PrescriptionMode"))  
       .withColumn("PRESCRIPTION_TYPE_RAW", col("PrescriptionType"))  
       .withColumn("PRESCRIPTION_FLG_RAW", col("PrescriptionIdent"))   
       .withColumn("REIMBURSEMENT_FLG_RAW", col("Reimbursement"))
       .withColumn("ISSUED_VOUCHERS_AMOUNT_RAW", col("ValueIssuedVouchers"))
       .withColumn("USED_VOUCHERS_AMOUNT_RAW", col("ValueUsedVouchers"))
       .withColumn("ISSUED_VOUCHERS_NUM_RAW", col("NumberIssuedVouchers"))
       .withColumn("USED_VOUCHERS_NUM_RAW", col("NumberUsedVouchers"))
       .withColumn("NUM_VALIDATED_FIELDS_COMPLETENESS",lit(__FMT_SL_NUM_VALIDATED_FIELDS_COMPLETENESS__).cast(IntegerType()))
       .withColumn("NUM_VALIDATED_FIELDS_ACCURACY",lit(__FMT_SL_NUM_VALIDATED_FIELDS_ACCURACY__).cast(IntegerType()))
       .withColumn("NUM_VALIDATED_FIELDS_DUPLICATE",lit(__FMT_SL_NUM_VALIDATED_FIELDS_DUPLICATE__).cast(IntegerType()))
       .withColumn("NUM_VALIDATED_FIELDS_CONFORMITY",lit(__FMT_SL_NUM_VALIDATED_FIELDS_CONFORMITY__).cast(IntegerType()))      
       .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", lit(0).cast(IntegerType()))
       .withColumn("NUM_ERROR_FIELDS_ACCURACY",  lit(0).cast(IntegerType()))
       .withColumn("NUM_ERROR_FIELDS_DUPLICATE",  lit(0).cast(IntegerType()))
       .withColumn("NUM_ERROR_FIELDS_CONFORMITY",lit(0).cast(IntegerType()))
              )

    ADP_log_debug(process, logger_name, level_action, log_level, "--After add raw columns", sys._getframe().f_code.co_name)
	
    df_detail = (df_detail
      # Validate OperationLine
      .withColumn("OperationLine_Err2",  when((isDigit("OperationLine")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None)))         
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("OperationLine")==False), col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))

      .withColumn("OperationLine_Err2",   
                  when(
                      ((length("OperationLine")==__OPERATIONLINE_LEN__)==False), 
                        (when(((length("OperationLine_Err2")==0)==True),('"' + 'DATA_LENGTH' + '":' + '"1"')).otherwise(concat(col("OperationLine_Err2"), lit(',"' + 'DATA_LENGTH' + '":' + '"1"'))))
                      ).otherwise(col("OperationLine_Err2"))                       
                 )
      .withColumn("OperationLine_Err2",  when(((length("OperationLine_Err2")>0)==True),concat(lit("{"),col("OperationLine_Err2"),lit("}"))).otherwise(lit(None)))
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((length("OperationLine")==__OPERATIONLINE_LEN__)==False, col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
                 
        # Validate OperationDate 
      .withColumn("OperationDate_Err2",  when( (udf_isDate_sql("OperationDate",lit(__YYYYMMDDhhmmss__)))==False, ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None))) 
      .withColumn("OperationDate_Err2",  when(((length("OperationDate_Err2")>0)==True),concat(lit("{"),col("OperationDate_Err2"),lit("}"))).otherwise(lit(None)))
                 
       .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when(udf_isDate_sql("OperationDate",lit(__YYYYMMDDhhmmss__))==False,\
               col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
                 
       # Validate TotalOfReceipt           
      .withColumn("TotalOfReceipt_Err2",   when((isDigit("TotalOfReceipt")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None))) 
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("TotalOfReceipt")==False), col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
                 
      .withColumn("TotalOfReceipt_Err2",   
                  when(
                      ((length("TotalOfReceipt")==0)==True),  
                        (when(((length("TotalOfReceipt_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("TotalOfReceipt_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("TotalOfReceipt_Err2"))                       
                 )
      .withColumn("TotalOfReceipt_Err2",   when(((length("TotalOfReceipt_Err2")>0)==True),concat(lit("{"),col("TotalOfReceipt_Err2"),lit("}"))).otherwise(lit(None)))  
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("TotalOfReceipt")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))
                 
       # Validate DiscountValOvertTotReceipt           
      .withColumn("DiscountValOvertTotReceipt_Err2",   when((isDigit("DiscountValOvertTotReceipt")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None))) 
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("DiscountValOvertTotReceipt")==False), col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))          
      .withColumn("DiscountValOvertTotReceipt_Err2",   
                  when(
                      ((length("DiscountValOvertTotReceipt")==0)==True),  
                        (when(((length("DiscountValOvertTotReceipt_Err2")==0)==True),
                              ('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("DiscountValOvertTotReceipt_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("DiscountValOvertTotReceipt_Err2"))                       
                 )
      .withColumn("DiscountValOvertTotReceipt_Err2",   when(((length("DiscountValOvertTotReceipt_Err2")>0)==True),concat(lit("{"),col("DiscountValOvertTotReceipt_Err2"),lit("}"))).otherwise(lit(None)))  
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("DiscountValOvertTotReceipt_Err2")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))
                 
       # Validate DecreasedValTotReceipt           
      .withColumn("DecreasedValTotReceipt_Err2", when((isDigit("DecreasedValTotReceipt")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None))) 
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("DecreasedValTotReceipt")==False), col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
                 
      .withColumn("DecreasedValTotReceipt_Err2",   when(((length("DecreasedValTotReceipt_Err2")>0)==True),concat(lit("{"),col("DecreasedValTotReceipt_Err2"),lit("}"))).otherwise(lit(None)))  

       # Validate ProductCode           
      .withColumn("ProductCode_Err2",   when(((length("ProductCode")==0)==True),  ('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(lit(None)))                
      .withColumn("ProductCode_Err2",   when(((length("ProductCode_Err2")>0)==True),concat(lit("{"),col("ProductCode_Err2"),lit("}"))).otherwise(lit(None)))  
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("ProductCode")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))

       # Validate ProductPacks             
      .withColumn("ProductPacks_Err2",   when((isDigit("ProductPacks")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None)))
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("ProductPacks")==False), col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
      .withColumn("ProductPacks_Err2",   
                  when(
                      ((length("ProductPacks")==0)==True),  
                        (when(((length("ProductPacks_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("ProductPacks_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("ProductPacks_Err2"))                       
                 )
      .withColumn("ProductPacks_Err2",   when(((length("ProductPacks_Err2")>0)==True),concat(lit("{"),col("ProductPacks_Err2"),lit("}"))).otherwise(lit(None))) 
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("ProductPacks")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))     

       # Validate ProductPackSize          
      .withColumn("ProductPackSize_Err2", when((isDigit("ProductPackSize")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None)))
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("ProductPackSize")==False), col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
      .withColumn("ProductPackSize_Err2",   
                  when(
                      ((length("ProductPackSize")==0)==True),  
                        (when(((length("ProductPackSize_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("ProductPackSize_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("ProductPackSize_Err2"))                       
                 )
      .withColumn("ProductPackSize_Err2", when(((length("ProductPackSize_Err2")>0)==True),concat(lit("{"),col("ProductPackSize_Err2"),lit("}"))).otherwise(lit(None)))                        
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("ProductPackSize")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS"))) 
                 
       # Validate ProductPriceCatalog  
      .withColumn("ProductPriceCatalog_Err2", when((isDigit("ProductPriceCatalog")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None)))
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("ProductPriceCatalog")==False), col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
      .withColumn("ProductPriceCatalog_Err2",   
                  when(
                      ((length("ProductPriceCatalog")==0)==True),  
                        (when(((length("ProductPriceCatalog_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("ProductPriceCatalog_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("ProductPriceCatalog_Err2"))                       
                 )
      .withColumn("ProductPriceCatalog_Err2",   when(((length("ProductPriceCatalog_Err2")>0)==True),concat(lit("{"),col("ProductPriceCatalog_Err2"),lit("}"))).otherwise(lit(None)))  
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("ProductPriceCatalog")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))
                 
       # Validate ProductPrice        
      .withColumn("ProductPrice_Err2", when((isDigit("ProductPrice")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None))) 
      .withColumn("ProductPrice_Err2",   
                  when(
                      ((length("ProductPrice")==0)==True),  
                        (when(((length("ProductPrice_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("ProductPrice_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("ProductPrice_Err2"))                       
                 )
      .withColumn("ProductPrice_Err2",  when(((length("ProductPrice_Err2")>0)==True),concat(lit("{"),col("ProductPrice_Err2"),lit("}"))).otherwise(lit(None)))
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("ProductPrice")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))

       # Validate DiscountValue
      .withColumn("DiscountValue_Err2", when((isDigit("DiscountValue")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None)))
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("DiscountValue")==False), col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
      .withColumn("DiscountValue_Err2", when(((length("DiscountValue_Err2")>0)==True),concat(lit("{"),col("DiscountValue_Err2"),lit("}"))).otherwise(lit(None)))  

       # Validate ProductTotalPrice                   
      .withColumn("ProductTotalPrice_Err2", when((isDigit("ProductTotalPrice")==False),  ('"' + 'DATA_TYPE' + '":' + '"1"')).otherwise(lit(None)))
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("ProductTotalPrice")==False), col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
      .withColumn("ProductTotalPrice_Err2",   
                  when(
                      ((length("ProductTotalPrice")==0)==True),  
                        (when(((length("ProductTotalPrice_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("ProductTotalPrice_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("ProductTotalPrice_Err2"))                       
                 )
      .withColumn("ProductTotalPrice_Err2", when(((length("ProductTotalPrice_Err2")>0)==True),concat(lit("{"),col("ProductTotalPrice_Err2"),lit("}"))).otherwise(lit(None)))             
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("ProductTotalPrice")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))
                 
       # Validate ReimbursementValue 
      .withColumn("ReimbursementValue_Err2", when((isDigit("ReimbursementValue")==True)| (length(col("ReimbursementValue"))==0), lit(None)).otherwise(('"' + 'DATA_TYPE' + '":' + '"1"')))
      .withColumn("ReimbursementValue_Err2",  when(((length("ReimbursementValue_Err2")>0)==True),concat(lit("{"),col("ReimbursementValue_Err2"),lit("}"))).otherwise(lit(None)))         

       # Validate ConsumerValue                 
      .withColumn("ConsumerValue_Err2", when((isDigit("ConsumerValue")==True)| (length(col("ConsumerValue"))==0), lit(None)).otherwise(('"' + 'DATA_TYPE' + '":' + '"1"')))
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when((isDigit("ConsumerValue")==True)| (length(col("ConsumerValue"))==0),col("NUM_ERROR_FIELDS_CONFORMITY")).\
                                                     otherwise( col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())))
      .withColumn("ConsumerValue_Err2",   
                  when(
                      ((length("ConsumerValue")==__CONSUMERVALUE_LEN__)==False),      
                        (when(((length("ConsumerValue_Err2")==0)==True),('"' + 'DATA_LENGTH' + '":' + '"1"')).otherwise(concat(col("ConsumerValue_Err2"), lit(',"' + 'DATA_LENGTH' + '":' + '"1"'))))
                      ).otherwise(col("ConsumerValue_Err2"))                       
                 )
      .withColumn("NUM_ERROR_FIELDS_CONFORMITY", when(((length("ConsumerValue")==__CONSUMERVALUE_LEN__)==False) & (length(col("ConsumerValue"))>0),  col("NUM_ERROR_FIELDS_CONFORMITY")+lit(1).cast(IntegerType())).\
                                                    otherwise(col("NUM_ERROR_FIELDS_CONFORMITY")))
       # Validate PaymentMode          
      .withColumn("PaymentMode_Err2", when((col("PaymentMode").isin(list(lov_PAYMENT_MODE.keys()))==False),  ('"' + 'DATA_LOV' + '":' + '"1"')).otherwise(lit(None)))
      .withColumn("NUM_ERROR_FIELDS_ACCURACY", when((col("PaymentMode").isin(list(lov_PAYMENT_MODE.keys()))==False), col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())).\
                                                    otherwise(col("NUM_ERROR_FIELDS_ACCURACY")))
      .withColumn("PaymentMode_Err2",   
                  when(
                      ((length("PaymentMode")==0)==True),  
                        (when(((length("PaymentMode_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("PaymentMode_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("PaymentMode_Err2"))
                 )
      .withColumn("PaymentMode_Err2",  when(((length("PaymentMode_Err2")>0)==True),concat(lit("{"),col("PaymentMode_Err2"),lit("}"))).otherwise(lit(None)))  
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("PaymentMode")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))               

       # Validate ConsumerGender  (nullable LOV)  
      .withColumn("ConsumerGender_Err2", when(((col("ConsumerGender").isin(list(lov_CONSUMER_GENDER.keys()))) | (length(col("ConsumerGender"))==0)), lit(None)).otherwise(('"' + 'DATA_LOV' + '":' + '"1"')))   
      .withColumn("NUM_ERROR_FIELDS_ACCURACY", when(((col("ConsumerGender").isin(list(lov_CONSUMER_GENDER.keys()))) | (length(col("ConsumerGender"))==0)), col("NUM_ERROR_FIELDS_ACCURACY")).\
                                                     otherwise(col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())))
      .withColumn("ConsumerGender_Err2", when(((length("ConsumerGender_Err2")>0)==True),concat(lit("{"),col("ConsumerGender_Err2"),lit("}"))).otherwise(lit(None)))   

       # Validate ConsumerType  (nullable)         
      .withColumn("ConsumerType_Err2", when((col("ConsumerType").isin(list(lov_CONSUMER_TYPE.keys())) | (length(col("ConsumerType"))==0)), lit(None)).otherwise(('"' + 'DATA_LOV' + '":' + '"1"')))           
      .withColumn("ConsumerType_Err2", when(((length("ConsumerType_Err2")>0)==True),concat(lit("{"),col("ConsumerType_Err2"),lit("}"))).otherwise(lit(None)))       
	  .withColumn("NUM_ERROR_FIELDS_ACCURACY", when((col("ConsumerType").isin(list(lov_CONSUMER_TYPE.keys())) | (length(col("ConsumerType"))==0)), col("NUM_ERROR_FIELDS_ACCURACY")).\
                                                    otherwise(col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())))

       # Validate OperationIdent   (required)        
      .withColumn("OperationIdent_Err2", when((col("OperationIdent").isin(list(lov_OPERATION_TYPE.keys()))==False),  ('"' + 'DATA_LOV' + '":' + '"1"')).otherwise(lit(None)))    
      .withColumn("NUM_ERROR_FIELDS_ACCURACY", when((col("OperationIdent").isin(list(lov_OPERATION_TYPE.keys()))==False), col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())).\
                                                    otherwise(col("NUM_ERROR_FIELDS_ACCURACY")))
      .withColumn("OperationIdent_Err2",   
                  when(
                      ((length("OperationIdent")==0)==True),  
                        (when(((length("OperationIdent_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("OperationIdent_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("OperationIdent_Err2"))                       
                 )
      .withColumn("OperationIdent_Err2",  when(((length("OperationIdent_Err2")>0)==True),concat(lit("{"),col("OperationIdent_Err2"),lit("}"))).otherwise(lit(None)))  
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("OperationIdent")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))
                 
       # Validate PrescriptionIdent   (required)        
      .withColumn("PrescriptionIdent_Err2", when((col("PrescriptionIdent").isin(list(lov_YES_NO.keys()))==False),  ('"' + 'DATA_LOV' + '":' + '"1"')).otherwise(lit(None)))
      .withColumn("NUM_ERROR_FIELDS_ACCURACY", when((col("PrescriptionIdent").isin(list(lov_YES_NO.keys()))==False), col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_ACCURACY")))
      .withColumn("PrescriptionIdent_Err2",   
                  when(
                      ((length("PrescriptionIdent")==0)==True),  
                        (when(((length("PrescriptionIdent_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("PrescriptionIdent_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("PrescriptionIdent_Err2"))                       
                 )
      .withColumn("PrescriptionIdent_Err2",  when(((length("PrescriptionIdent_Err2")>0)==True),concat(lit("{"),col("PrescriptionIdent_Err2"),lit("}"))).otherwise(lit(None)))  
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("PrescriptionIdent")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))
                 
       # Validate PrescriptionMode  (nullable)         
      .withColumn("PrescriptionMode_Err2", when((col("PrescriptionMode").isin(list(lov_PRESCRIPTION_MODE.keys()))) | (length(col("PrescriptionMode"))==0), lit(None)).otherwise(('"' + 'DATA_LOV' + '":' + '"1"')))  
      .withColumn("NUM_ERROR_FIELDS_ACCURACY", when((col("PrescriptionMode").isin(list(lov_PRESCRIPTION_MODE.keys()))) | (length(col("PrescriptionMode"))==0),col("NUM_ERROR_FIELDS_ACCURACY")).\
                                                 otherwise(col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())))
      .withColumn("PrescriptionMode_Err2", when(((length("PrescriptionMode_Err2")>0)==True),concat(lit("{"),col("PrescriptionMode_Err2"),lit("}"))).otherwise(lit(None)))   

       # Validate PrescriptionType  (nullable)         
      .withColumn("PrescriptionType_Err2", when((col("PrescriptionType").isin(list(lov_PRESCRIPTION_TYPE.keys()))) | (length(col("PrescriptionType"))==0), lit(None)).otherwise(('"' + 'DATA_LOV' + '":' + '"1"'))) 
      .withColumn("NUM_ERROR_FIELDS_ACCURACY", when((col("PrescriptionType").isin(list(lov_PRESCRIPTION_TYPE.keys()))) | (length(col("PrescriptionType"))==0), col("NUM_ERROR_FIELDS_ACCURACY")).\
                                                otherwise(col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())))

      .withColumn("PrescriptionType_Err2", when(((length("PrescriptionType_Err2")>0)==True),concat(lit("{"),col("PrescriptionType_Err2"),lit("}"))).otherwise(lit(None)))   

       # Validate Reimbursement   (required)        
      .withColumn("Reimbursement_Err2", when((col("Reimbursement").isin(list(lov_YES_NO.keys()))==False),  ('"' + 'DATA_LOV' + '":' + '"1"')).otherwise(lit(None)))    
      .withColumn("NUM_ERROR_FIELDS_ACCURACY", when((col("Reimbursement").isin(list(lov_YES_NO.keys()))==False), col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_ACCURACY")))

      .withColumn("Reimbursement_Err2",   
                  when(
                      ((length("Reimbursement")==0)==True),  
                        (when(((length("Reimbursement_Err2")==0)==True),('"' + 'DATA_EMPTY' + '":' + '"1"')).otherwise(concat(col("Reimbursement_Err2"), lit(',"' + 'DATA_EMPTY' + '":' + '"1"'))))
                      ).otherwise(col("Reimbursement_Err2"))                       
                 )
      .withColumn("Reimbursement_Err2",  when(((length("Reimbursement_Err2")>0)==True),concat(lit("{"),col("Reimbursement_Err2"),lit("}"))).otherwise(lit(None)))  
      .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when((length("Reimbursement")==0)==True, col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())).otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")))
                )

    ADP_log_debug(process, logger_name, level_action, log_level, "--After Validations", sys._getframe().f_code.co_name)
    ################################################################################################################################################
    #  END VALIDATION FORMAT DATA FIELDS
    ################################################################################################################################################

    ################################################################################################################################################
    #  TRANSFORMATION DATA FIELDS
    ################################################################################################################################################

    df_detail = (df_detail     
                  #Numeric Required fields: when the field has a incorrect value OR a NULL value, it is replaced by the defect value -9999999
                 .withColumn("FILE_LINE_NUM", (when((isDigit("FILE_LINE_NUM")==False),  lit(-999)).otherwise(col("FILE_LINE_NUM").cast(DecimalType(10,0)))))
                 .withColumn("OperationLine", (when((isDigit("OperationLine")==False),  lit(-999)).otherwise(col("OperationLine").cast(DecimalType(10,0)))))
                 .withColumn("ProductPacks", (when((isDigit("ProductPacks")==False),  lit(-999999)).otherwise(col("ProductPacks").cast(DecimalType(10,0)))))
                 .withColumn("TotalOfReceipt", ((when((isDigit("TotalOfReceipt")==False),  lit(-999999)).otherwise(col("TotalOfReceipt").cast(IntegerType())/100))).cast(DecimalType(12,2)))    
                 .withColumn("ProductPrice", ((when((isDigit("ProductPrice")==False),  lit(-999999)).otherwise(col("ProductPrice").cast(IntegerType())/100))).cast(DecimalType(12,2)))          
                 .withColumn("ProductTotalPrice", ((when((isDigit("ProductTotalPrice")==False),  lit(-999999)).otherwise(col("ProductTotalPrice").cast(IntegerType())/100))).cast(DecimalType(12,2)))      

                  #Numeric Optional fields: when the field has a incorrect value, it is replaced by the defect value -9999999; when the field has NULL value it is replaced by 0 VALUE
                 .withColumn("ConsumerAge", (when((length(col("ConsumerAge"))==0), lit(0))\
                                                      .otherwise(when((isDigit("ConsumerAge")==False),  lit(-9999)).otherwise(col("ConsumerAge")))).cast(DecimalType(4,0)))
                 .withColumn("ProductPackSize", (when((length(col("ProductPackSize"))==0), lit(0))\
                                                      .otherwise(when((isDigit("ProductPackSize")==False),  lit(-999999)).otherwise(col("ProductPackSize")))).cast(DecimalType(10,0)))
                 .withColumn("NumberIssuedVouchers", (when((length(col("NumberIssuedVouchers"))==0), lit(0))\
                                                      .otherwise(when((isDigit("NumberIssuedVouchers")==False),  lit(-999999)).otherwise(col("NumberIssuedVouchers")))).cast(DecimalType(10,0)))
                 .withColumn("NumberUsedVouchers", (when((length(col("NumberUsedVouchers"))==0), lit(0))\
                                                      .otherwise(when((isDigit("NumberUsedVouchers")==False),  lit(-999999)).otherwise(col("NumberUsedVouchers")))).cast(DecimalType(10,0)))
                 .withColumn("ValueIssuedVouchers", ((when((length(col("ValueIssuedVouchers"))==0), lit(0.0)).otherwise(when((isDigit("ValueIssuedVouchers")==False),  lit(-999999))\
                                                                                                             .otherwise(col("ValueIssuedVouchers")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("ValueUsedVouchers", ((when((length(col("ValueUsedVouchers"))==0), lit(0.0)).otherwise(when((isDigit("ValueUsedVouchers")==False),  lit(-999999))\
                                                                                                         .otherwise(col("ValueUsedVouchers")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("ProductPriceCatalog", ((when((length(col("ProductPriceCatalog"))==0), lit(0.0)).otherwise(when((isDigit("ProductPriceCatalog")==False),  lit(-999999))\
                                                                                                              .otherwise(col("ProductPriceCatalog")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("DiscountValOvertTotReceipt", ((when((length(col("DiscountValOvertTotReceipt"))==0), lit(0.0)).otherwise(when((isDigit("DiscountValOvertTotReceipt")==False),  lit(-999999))\
                                                                                                               .otherwise(col("DiscountValOvertTotReceipt")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("DecreasedValTotReceipt", ((when((length(col("DecreasedValTotReceipt"))==0), lit(0.0)).otherwise(when((isDigit("DecreasedValTotReceipt")==False),  lit(-999999))\
                                                                                                                   .otherwise(col("DecreasedValTotReceipt")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("DiscountValue", ((when((length(col("DiscountValue"))==0), lit(0.0)).otherwise(when((isDigit("DiscountValue")==False),  lit(-999999))\
                                                                                                 .otherwise(col("DiscountValue")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("ProductFee", ((when((length(col("ProductFee"))==0), lit(0.0)).otherwise(when((isDigit("ProductFee")==False),  lit(-999999))\
                                                                                           .otherwise(col("ProductFee")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("ProductMarkup", ((when((length(col("ProductMarkup"))==0), lit(0.0)).otherwise(when((isDigit("ProductMarkup")==False),  lit(-999999))\
                                                                                                 .otherwise(col("ProductMarkup")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("CoPaymentValue", ((when((length(col("CoPaymentValue"))==0), lit(0.0)).otherwise(when((isDigit("CoPaymentValue")==False),  lit(-999999))\
                                                                                                 .otherwise(col("CoPaymentValue")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("ReimbursementValue", ((when((length(col("ReimbursementValue"))==0),  lit(0.0)).otherwise(when((isDigit("ReimbursementValue")==False),  lit(-999999))\
                                                                                                            .otherwise(col("ReimbursementValue")))).cast(IntegerType())/100).cast(DecimalType(12,2)))
                 .withColumn("ConsumerValue", ((when((length(col("ConsumerValue"))==0), lit(0.0)).otherwise(when((isDigit("ConsumerValue")==False),  lit(-999999))\
                                                                                                            .otherwise(col("ConsumerValue")))).cast(IntegerType())/100).cast(DecimalType(12,2)))

                  #Required (when the field has a incorrect value OR a NULL value, it is replaced by the defect value)
                 .withColumn("OperationDate",  when((udf_isDate_sql("OperationDate",lit(__YYYYMMDDhhmmss__))==True), col("OperationDate")).otherwise("00010101000000"))              
                 .withColumn("PaymentMode", when((col("PaymentMode").isin(list(lov_PAYMENT_MODE.keys()))==True), col("PaymentMode")).otherwise("-1"))
                 .withColumn("OperationIdent", when((col("OperationIdent").isin(list(lov_OPERATION_TYPE.keys()))==True), col("OperationIdent")).otherwise("-1"))                 
                 .withColumn("ProductCode",  when(((length("ProductCode")==0)==False), col("ProductCode")).otherwise("-9"))

                  #Optional (when the field has a Null value it does not replaced; when the field has a incorrect value it is replaced by the defect value)
                 .withColumn("ConsumerGender", when(((col("ConsumerGender").isin(list(lov_CONSUMER_GENDER.keys()))==True) | (length(col("ConsumerGender"))==0)), col("ConsumerGender")).otherwise("-1"))
                 .withColumn("ConsumerType", when(((col("ConsumerType").isin(list(lov_CONSUMER_TYPE.keys()))==True) | (length(col("ConsumerType"))==0)), col("ConsumerType")).otherwise("-1"))
                 .withColumn("PrescriptionMode", when(((col("PrescriptionMode").isin(list(lov_PRESCRIPTION_MODE.keys()))==True) | (length(col("PrescriptionMode"))==0)), col("PrescriptionMode")).otherwise("-1"))
                 .withColumn("PrescriptionType", when(((col("PrescriptionType").isin(list(lov_PRESCRIPTION_TYPE.keys()))==True) | (length(col("PrescriptionType"))==0)), col("PrescriptionType")).otherwise("-1"))
                 .withColumn("PrescriptionIdent", when(((col("PrescriptionIdent").isin(list(lov_YES_NO.keys()))==True) | (length(col("PrescriptionIdent"))==0)), col("PrescriptionIdent")).otherwise("-"))
                 .withColumn("Reimbursement", when(((col("Reimbursement").isin(list(lov_YES_NO.keys()))==True) | (length(col("Reimbursement"))==0)), col("Reimbursement")).otherwise("-"))

                   #Transform to TimeStamp or Date 
                 .withColumn("FileDate",   to_timestamp("FileDate", "yyyyMMddHHmmss"))
                 .withColumn("LANDING_DATE",   to_timestamp("LANDING_DATE", "yyyyMMddHHmmss"))
                 .withColumn("PROCESS_DATE",   to_timestamp("PROCESS_DATE", "yyyyMMddHHmmss"))
                 .withColumn("OperationDate",   to_timestamp("OperationDate", "yyyyMMddHHmmss"))

                  #Transform the negative TotalOfReceipt
                 .withColumn("TotalOfReceipt", (when(col('TotalReceiptSignal')==lit('N'),  (col("TotalOfReceipt")*-1)).otherwise(col("TotalOfReceipt"))).cast(DecimalType(12,2)) ) 
              ).alias('df_detail')
    ADP_log_debug(process, logger_name, level_action, log_level, "--After Transformations - Default values", sys._getframe().f_code.co_name)
   
    #Define window to aggregate amounts at operation level
    windowSpec = Window.partitionBy(df_detail['FILE_NAME'], df_detail['PharmacyID'], df_detail['OperationDate'], df_detail['OperationID'])

    df_detail = (df_detail
                 .withColumn("SUM_ProductTotalPrice",sum("ProductTotalPrice").over(windowSpec))
                 .withColumn("SUM_ProductTotalPrice", when((col('SUM_ProductTotalPrice').isNull()==True), lit(0.0)).otherwise(col('SUM_ProductTotalPrice')))
                 
                 .withColumn("PRODUCT_NET_PRICE",(col('ProductTotalPrice') / col('ProductPacks')))
                 .withColumn("PRODUCT_NET_PRICE", when((col('PRODUCT_NET_PRICE').isNull()==True), lit(0.0)).otherwise(col('PRODUCT_NET_PRICE')))
                 .withColumn("DISCOUNT_WEIGTHED_AMOUNT",((col('DiscountValOvertTotReceipt') / col('SUM_ProductTotalPrice')) * col('ProductTotalPrice')))
                 .withColumn("DISCOUNT_WEIGTHED_AMOUNT", when((col('DISCOUNT_WEIGTHED_AMOUNT').isNull()==True), lit(0.0)).otherwise(col('DISCOUNT_WEIGTHED_AMOUNT')))
                 .withColumn("PRODUCT_NET_WEIGTHED_PRICE",(col('PRODUCT_NET_PRICE')-col('DISCOUNT_WEIGTHED_AMOUNT')/col('ProductPacks')))
                 .withColumn("PRODUCT_NET_WEIGTHED_PRICE", when((col('PRODUCT_NET_WEIGTHED_PRICE').isNull()==True), lit(0.0)).otherwise(col('PRODUCT_NET_WEIGTHED_PRICE')))

                 .withColumn("PRODUCT_NET_PRICE",(col('PRODUCT_NET_PRICE')).cast(DecimalType(12,2)))
                 .withColumn("DISCOUNT_WEIGTHED_AMOUNT",(col('DISCOUNT_WEIGTHED_AMOUNT')).cast(DecimalType(12,2)))
                 .withColumn("PRODUCT_NET_WEIGTHED_PRICE",(col('PRODUCT_NET_WEIGTHED_PRICE')).cast(DecimalType(12,2)))        
                ).alias('df_detail')
    ADP_log_debug(process, logger_name, level_action, log_level, "--After Transformations - Calculation of amounts", sys._getframe().f_code.co_name)

    ################################################################################################################################################
    #  END - TRANSFORMATION DATA FIELDS
    ################################################################################################################################################
    ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name)
	
    return df_detail
  
  except Exception as err:
    ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
    raise Exception(err)

#############################################################################################################################

def EnrichmentFarmaticSellout(df,debug=__DEBUG_DEFAULT__,partitions=__PARTITIONS_DEFAULT__):
  """Enrich Sellout Columns structure and Add a Column for each field with the validation Result

    Parameters:
      df                   -- Dataframe with the result of the FieldFormatValidations Process
      debug                -- True for enable debug verbosing or False to not enable
      partitions           -- Partitions defined

    Return:
      Dataframe            -- Dataframe containing output columns from FieldFormatValidations and the new enrichment columns
  """
  #Who                 When           What
  #Victor Salesa       18/12/2018     Create Function Stub
  #Ana Perez           21/12/2018     Enrichment from Master Data
  #Ana Perez           25/01/2019     Using the new canonical masterdata format
  #Ana Perez           05/02/2019     Takes the first row by PRODUCT_INTERNAL_CODE and EAN13 (without duplicated rows) from m_product_cat
  #Ana Perez           14/02/2019     Add new persist and print lines
  #Ana Perez           19/02/2019     Fix error in categorization enrichment process
  #Ana Perez           04/03/2019     Unpersist dataframes doesent used
  #Ana Perez           10/04/2019     Included log managment and exception managment
  try:    
    
    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)
    df_detail= df.alias('df_detail')

    ################################################################################################################################################
    #  ENRICHMENT DATA FIELDS
    #                  this process uses alias so that the fields maintain compatibility with previous versions
    ################################################################################################################################################

     #Enrichment with PRODUCT MASTER DATA
    ADP_log_debug(process, logger_name, level_action, log_level, "[EnrichmentFarmaticSellout: Begin Enrichment PRODUCT Categories]. " + str(df_detail.count()) + " rows", sys._getframe().f_code.co_name)  
    df_product_cat = (spark.table(__CATEGORIES_TABLE_NAME__)    
                      .withColumn("NATIONAL_CODE", col('PRODUCT_INTERNAL_CODE').substr(0,6))
                      .filter((col('NATIONAL_CODE')> "149999"))
                      .select(col('PRODUCT_INTERNAL_CODE')
                          , col('NATIONAL_CODE')
                          , col('BAR_CODE').alias('EAN13')
                          , col('LEGAL_CATEGORY').alias('CLASS')
                          , col('COMMERCIAL_CATEGORY_L1').alias('CATEGORY')
                          , col('COMMERCIAL_CATEGORY_L2').alias('FAMILY')
                          , col('COMMERCIAL_CATEGORY_L3').alias('SUBFAMILY')
                          , col('BRAND').alias('BRAND')
                          , col('MANUFACTURER_CODE').alias('LABORATORY_CODE')
                          , col('MANUFACTURER_NAME').alias('LABORATORY_NAME')))

     #takes the first row by PRODUCT_INTERNAL_CODE and EAN13 (without duplicated rows)
    windowSpec = Window.partitionBy(df_product_cat['PRODUCT_INTERNAL_CODE']).orderBy(col('EAN13').desc())
    
    df_product_cat = (df_product_cat
                      .withColumn("EAN13_filter", when(col('EAN13').isNull()==True, lit('x')).otherwise(col('EAN13')))
                      .withColumn("First_EAN13", first("EAN13_filter").over(windowSpec) )
                      .filter(col('EAN13_filter')==col('First_EAN13'))
                      .withColumn("CLASS", when(((col('CLASS')=='PENDIENTE ASIGNAR') | (col('CLASS')=='SIN CLASE')),lit(None)).otherwise(col('CLASS')))
                      .withColumn("CATEGORY", when(((col('CATEGORY')=='PENDIENTE ASIGNAR') | (col('CATEGORY')=='SIN CATEGORÍA') | (col('CATEGORY')=='SIN CATEGORIA')),lit(None)).otherwise(col('CATEGORY')))
                      .withColumn("FAMILY", when(((col('FAMILY')=='PENDIENTE ASIGNAR') | (col('FAMILY')=='SIN FAMILIA')),lit(None)).otherwise(col('FAMILY')))			
                      .withColumn("SUBFAMILY", when(((col('SUBFAMILY')=='PENDIENTE ASIGNAR') | (col('SUBFAMILY')=='SIN SUBFAMILIA')),lit(None)).otherwise(col('SUBFAMILY')))
                      .withColumn("BRAND", when(((col('BRAND')=='PENDIENTE ASIGNAR') | (col('BRAND')=='SIN TIPO')),lit(None)).otherwise(col('BRAND')))
                      .withColumn("LABORATORY_NAME", when(((col('LABORATORY_NAME')=='PENDIENTE ASIGNAR') | (col('LABORATORY_NAME')=='SIN TIPO')),lit(None)).otherwise(col('LABORATORY_NAME')))
                      .drop("First_EAN13")
                      .distinct()
                     ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_product_cat')
    df_product_cat.count()

    
    ADP_log_debug(process, logger_name, level_action, log_level, "[Readed Product Cat Unique row by PRODUCT_INTERNAL_CODE and EAN]", sys._getframe().f_code.co_name)  
    
    #Products with EAN and NATIONAL CODE  
    df_product = (df_product_cat
                    .filter((df_product_cat.EAN13.isNull() == False))
                    .distinct()
                 ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_product')
    df_product.count()
    ADP_log_debug(process, logger_name, level_action, log_level, "[Filtered if National and EAN Not null]", sys._getframe().f_code.co_name)
    
    #Joing Products with EAN and NATIONAL CODE with Sell Out 
    df_detail = (df_detail.repartition(partitions)
                   .join(df_product.repartition(partitions), ((col('df_detail.EANCode')==col('df_product.EAN13'))
                                                     & (col('df_detail.ProductCode')==col('df_product.NATIONAL_CODE')) ), how='left')
                   .select('df_detail.*'
                          , 'df_product.NATIONAL_CODE'
                          , 'df_product.EAN13'
                          , 'df_product.CLASS'
                          , 'df_product.CATEGORY'
                          , 'df_product.FAMILY'
                          , 'df_product.SUBFAMILY'
                          , 'df_product.BRAND'
                          , 'df_product.LABORATORY_NAME'
                          , 'df_product.LABORATORY_CODE'
                          )
                ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_detail')
    df_detail.count()
    
    ADP_log_debug(process, logger_name, level_action, log_level, "[Joined PMS and product_cat filtered by EAN not null] Rows: " + str(df_detail.count()) + " rows", sys._getframe().f_code.co_name)  
	
     #Select Product Cat, unique row and  Without EAN 
    df_product2 = (df_product_cat
                  .select( col('NATIONAL_CODE')
                          , col('CLASS')
                          , col('CATEGORY')
                          , col('FAMILY')
                          , col('SUBFAMILY')
                          , col('BRAND')
                          , col('LABORATORY_NAME')
                          , col('LABORATORY_CODE')
                          )
                  .distinct())
    
    df_product2 = df_product2.persist(StorageLevel.MEMORY_AND_DISK).alias('df_product2')
    df_product2.count()
    
    ADP_log_debug(process, logger_name, level_action, log_level, "[Selected Product Cat, unique row and  Without EAN ] Rows: " + str(df_detail.count()) + " rows", sys._getframe().f_code.co_name)  
      
    df_detail = (df_detail.repartition(partitions).join(df_product2.repartition(partitions), col('df_detail.ProductCode')==col('df_product2.NATIONAL_CODE'), how='left')
                  .select('df_detail.*'
                          , col('df_product2.NATIONAL_CODE').alias('dp_NATIONAL_CODE')
                          , col('df_product2.CLASS').alias('dp_CLASS')
                          , col('df_product2.CATEGORY').alias('dp_CATEGORY')
                          , col('df_product2.FAMILY').alias('dp_FAMILY')
                          , col('df_product2.SUBFAMILY').alias('dp_SUBFAMILY')
                          , col('df_product2.BRAND').alias('dp_BRAND')
                          , col('df_product2.LABORATORY_NAME').alias('dp_LABORATORY_NAME')
                          , col('df_product2.LABORATORY_CODE').alias('dp_LABORATORY_CODE')
                          )
                  .withColumn("NATIONAL_CODE",
                              when((col('df_detail.NATIONAL_CODE').isNull()==True)  & (col('dp_NATIONAL_CODE').isNull()==False), col('dp_NATIONAL_CODE')).otherwise(col('df_detail.NATIONAL_CODE')))              
                  .withColumn("CLASS",
                              when((col('df_detail.CLASS').isNull()==True) & (col('dp_CLASS').isNull()==False) , col('dp_CLASS')).otherwise(col('df_detail.CLASS')))         
                  .withColumn("CATEGORY",       
                         when((col('df_detail.CATEGORY').isNull()==True) & (col('dp_CATEGORY').isNull()==False) , col('dp_CATEGORY')).otherwise(col('df_detail.CATEGORY')))               
                  .withColumn("FAMILY",    
                       when((col('df_detail.FAMILY').isNull()==True) & (col('dp_FAMILY').isNull()==False) , col('dp_FAMILY')).otherwise(col('df_detail.FAMILY'))) 
                  .withColumn("SUBFAMILY",    
                       when((col('df_detail.SUBFAMILY').isNull()==True) & (col('dp_SUBFAMILY').isNull()==False) , col('dp_SUBFAMILY')).otherwise(col('df_detail.SUBFAMILY')))   
                  .withColumn("BRAND",    
                       when((col('df_detail.BRAND').isNull()==True) & (col('dp_BRAND').isNull()==False) , col('dp_BRAND')).otherwise(col('df_detail.BRAND'))) 
                  .withColumn("LABORATORY_NAME",    
                       when((col('df_detail.LABORATORY_NAME').isNull()==True) & (col('dp_LABORATORY_NAME').isNull()==False) , col('dp_LABORATORY_NAME')).otherwise(col('df_detail.LABORATORY_NAME')))
                  .withColumn("LABORATORY_CODE",    
                       when((col('df_detail.LABORATORY_CODE').isNull()==True) & (col('dp_LABORATORY_CODE').isNull()==False) , col('dp_LABORATORY_CODE')).otherwise(col('df_detail.LABORATORY_CODE')))
                  .drop(col('dp_NATIONAL_CODE'))
                  .drop(col('dp_EAN13'))
                  .drop(col('dp_CLASS'))
                  .drop(col('dp_CATEGORY'))
                  .drop(col('dp_FAMILY'))
                  .drop(col('dp_SUBFAMILY'))
                  .drop(col('dp_BRAND'))
                  .drop(col('dp_LABORATORY_NAME'))
                  .drop(col('dp_LABORATORY_CODE'))
              ).alias('df_detail')


    #Assign Free products (NATIONAL_CODE -9), NATIONAL_CODE -1
    df_detail= (df_detail
                   .withColumn("NATIONAL_CODE", 
                           when((col('ProductCode')< "150000") & (col('NATIONAL_CODE').isNull()==True),"-9").otherwise( col('NATIONAL_CODE')))              
                   .withColumn("NATIONAL_CODE",
                           when((col('NATIONAL_CODE').isNull()==False), col('NATIONAL_CODE')).otherwise("-1"))
                 ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_detail')
    df_detail.count()
    
    ADP_log_debug(process, logger_name, level_action, log_level, "[Joined Sell Out with product cat without relation by EAN] Rows:  " + str(df_detail.count()) + " rows", sys._getframe().f_code.co_name)
      
    ########################### ENRICH PHARMACIES
    ADP_log_debug(process, logger_name, level_action, log_level, "[Begin Enrichment PHARMACIES]", sys._getframe().f_code.co_name)
        
    df_pharmacy = (   spark.table(__PHARMACY_TABLE_NAME__)
                      .select(col('PHARMACY_ADHERED_CODE').alias('PH_AH_PHARMACY_CODE')
                           ,col('PHARMACY_CUSTOMER_CODE'))
                      .filter(length(col('PHARMACY_ADHERED_CODE'))!=0)
                      .distinct()
                     ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_pharmacy')
    df_pharmacy.count()

    df_detail = (df_detail
                 .join(broadcast(df_pharmacy), (col('df_detail.PharmacyID')==col('df_pharmacy.PH_AH_PHARMACY_CODE')), how='left')
                 .select ('df_detail.*'
                          , col('df_pharmacy.PHARMACY_CUSTOMER_CODE').alias('CUSTOMER_CODE'))
                 .withColumn("CUSTOMER_CODE",
                           when((col('CUSTOMER_CODE').isNull()==False), col('CUSTOMER_CODE')).otherwise("-1"))
                 ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_detail')

    df_detail.count()
    ADP_log_debug(process, logger_name, level_action, log_level, "[Read Pharmacies and Join Sell-out] Rows:  " + str(df_detail.count()) + " rows", sys._getframe().f_code.co_name)
	
    ########################### ENRICH PRODUCT-MANUFACTURER and PRODUCT_NAME
    ADP_log_debug(process, logger_name, level_action, log_level, "[Begin Enrich MANUFACTURERS]", sys._getframe().f_code.co_name)
	
     #Pharmacies with MANUFACTURER_CODE
    df_product_mdm = (spark.table(__PRODUCT_TABLE_NAME__)
                      .select(col('PRODUCT_INTERNAL_CODE').alias('PRODUCT_CODE')
                               ,col('MANUFACTURER_CODE')
                               ,col('PRODUCT_NAME'))
                      .withColumn("mdm_PRODUCT_CODE",  col('PRODUCT_CODE').substr(0,6))   #it takes the manufacturer_code finished  at "0"
                      .distinct()
                      .filter((col('mdm_PRODUCT_CODE')> "149999"))
                     ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_product_mdm')
    df_product_mdm.count()
    ADP_log_debug(process, logger_name, level_action, log_level, "[Read master data product and filter it the NOT free codes]", sys._getframe().f_code.co_name)
	
    df_detail = (df_detail
                 .join(broadcast(df_product_mdm), (col('df_detail.ProductCode')==df_product_mdm.mdm_PRODUCT_CODE), how='left')
                 .select ('df_detail.*'
                          , col('df_product_mdm.mdm_PRODUCT_CODE')
                          , col('df_product_mdm.MANUFACTURER_CODE')
                          , col('df_product_mdm.PRODUCT_NAME')
                          )
                 . withColumn("MANUFACTURER_CODE",                         
                           when((col('MANUFACTURER_CODE').isNull()==False), concat((col('MANUFACTURER_CODE').substr(0,5)),lit("0"))).otherwise("-1")) 
                 . withColumn("NATIONAL_CODE",                  #if national_code is not at mdm_categories file, and it is at mdm_item fields, it takes the mdm_item value      
                           when(((col('mdm_PRODUCT_CODE').isNull()==False) & (col('NATIONAL_CODE')=="-1")), col('mdm_PRODUCT_CODE')).otherwise(col('NATIONAL_CODE')))
                 ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_detail')
    df_detail.count()
    ADP_log_debug(process, logger_name, level_action, log_level, "[Joined Sell out with Master data product] Rows:  " + str(df_detail.count()) + " rows", sys._getframe().f_code.co_name)
	
    df_manufacturers_mdm = (spark.table(__MANUFACTURERS_TABLE_NAME__)
                          .select(col('MANUFACTURER_CODE')
                                 ,col('MANUFACTURER_NAME')
                                 )
                           ).persist(StorageLevel.MEMORY_AND_DISK).alias ('df_manufacturers_mdm')  
    df_manufacturers_mdm.count()
    ADP_log_debug(process, logger_name, level_action, log_level, "[Read master data manufacturers]", sys._getframe().f_code.co_name)
	
    df_detail = (df_detail
                 .join(broadcast(df_manufacturers_mdm), (col('df_detail.MANUFACTURER_CODE')== df_manufacturers_mdm.MANUFACTURER_CODE), how='left')
                 .select ('df_detail.*'
                          , col('df_manufacturers_mdm.MANUFACTURER_NAME'))
                 ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_detail') 
    df_detail.count()
    ADP_log_debug(process, logger_name, level_action, log_level, "[Joined Sell-Out with Manufacturs ] Rows:  " + str(df_detail.count()) + " rows", sys._getframe().f_code.co_name)
    
    df_detail = (df_detail
                 
                   .withColumn("MANUFACTURER_NAME",         #if the product mdm has not the manufacturer it take this data from categorization file
                             when((col('MANUFACTURER_NAME').isNull()==True), col('LABORATORY_NAME')).otherwise(col('MANUFACTURER_NAME')))
                   .withColumn("MANUFACTURER_CODE",         #if the product mdm has not the manufacturer it take this data from categorization file 
                             when((col('MANUFACTURER_CODE')=="-1"), col('LABORATORY_CODE')).otherwise(col('MANUFACTURER_CODE')))
                 ).alias('df_detail')   
    
    #################### DEFAULT VALUES ##########################################################################################################
    #Assign Free products (NATIONAL_CODE -9), NATIONAL_CODE -1, and default categorization values
    df_detail = (df_detail

                   .withColumn("PRODUCT_NAME",   
                           when(((col('PRODUCT_NAME').isNull()==True) & (col('NATIONAL_CODE')=="-1")), lit('UNKNOWN PRODUCT')).otherwise(col('PRODUCT_NAME')))
                   .withColumn("PRODUCT_NAME",   
                           when(((col('PRODUCT_NAME').isNull()==True) & (col('NATIONAL_CODE')=="-9")), lit('NA-FREE CODE')).otherwise(col('PRODUCT_NAME')))
                   .withColumn("PRODUCT_NAME",   
                           when(((col('PRODUCT_NAME').isNull()==True) & (col('NATIONAL_CODE')!="-9") & (col('NATIONAL_CODE')!="-1")), lit('UNKNOWN')).otherwise(col('PRODUCT_NAME')))

                   .withColumn("CLASS", when(((col('CLASS').isNull()==True) & (col('NATIONAL_CODE')=="-1")),lit('UNKNOWN PRODUCT')).otherwise(col('CLASS')))
                   .withColumn("CLASS", when(((col('CLASS').isNull()==True) & (col('NATIONAL_CODE')=="-9")),lit('NA-FREE CODE')).otherwise(col('CLASS')))
                   .withColumn("CLASS", when(((col('CLASS').isNull()==True) & (col('NATIONAL_CODE')>"0")),lit('UNKNOWN')).otherwise(col('CLASS')))

                   .withColumn("CATEGORY", when(((col('CATEGORY').isNull()==True) & (col('NATIONAL_CODE')=="-1")),lit('UNKNOWN PRODUCT')).otherwise(col('CATEGORY')))
                   .withColumn("CATEGORY", when(((col('CATEGORY').isNull()==True) & (col('NATIONAL_CODE')=="-9")),lit('NA-FREE CODE')).otherwise(col('CATEGORY')))
                   .withColumn("CATEGORY", when(((col('CATEGORY').isNull()==True) & (col('NATIONAL_CODE')>"0")),lit('UNKNOWN')).otherwise(col('CATEGORY')))

                   .withColumn("FAMILY", when(((col('FAMILY').isNull()==True) & (col('NATIONAL_CODE')=="-1")),lit('UNKNOWN PRODUCT')).otherwise(col('FAMILY')))
                   .withColumn("FAMILY", when(((col('FAMILY').isNull()==True) & (col('NATIONAL_CODE')=="-9")),lit('NA-FREE CODE')).otherwise(col('FAMILY')))

                   .withColumn("SUBFAMILY", when(((col('SUBFAMILY').isNull()==True) & (col('NATIONAL_CODE')=="-1")),lit('UNKNOWN PRODUCT')).otherwise(col('SUBFAMILY')))
                   .withColumn("SUBFAMILY", when(((col('SUBFAMILY').isNull()==True) & (col('NATIONAL_CODE')=="-9")),lit('NA-FREE CODE')).otherwise(col('SUBFAMILY')))

                   .withColumn("BRAND", when(((col('BRAND').isNull()==True) & (col('NATIONAL_CODE')=="-1")),lit('UNKNOWN PRODUCT')).otherwise(col('BRAND')))
                   .withColumn("BRAND", when(((col('BRAND').isNull()==True) & (col('NATIONAL_CODE')=="-9")),lit('NA-FREE CODE')).otherwise(col('BRAND')))
                   .withColumn("BRAND", when(((col('BRAND').isNull()==True) & (col('NATIONAL_CODE')>"0")),lit('UNKNOWN')).otherwise(col('BRAND')))
                   .withColumn("BRAND", (col('BRAND').substr(0,50)))

                   .withColumn("MANUFACTURER_NAME",
                           when(((col('MANUFACTURER_NAME').isNull()==True) & (col('NATIONAL_CODE')=="-1")),lit('UNKNOWN PRODUCT')).otherwise(col('MANUFACTURER_NAME')))
                   .withColumn("MANUFACTURER_NAME",
                           when(((col('MANUFACTURER_NAME').isNull()==True) & (col('NATIONAL_CODE')=="-9")),lit('NA-FREE CODE')).otherwise(col('MANUFACTURER_NAME')))
                   .withColumn("MANUFACTURER_NAME", when(((col('MANUFACTURER_NAME').isNull()==True) & (col('NATIONAL_CODE')>"0")),lit('UNKNOWN')).otherwise(col('MANUFACTURER_NAME')))

                   .withColumn("MANUFACTURER_CODE", when(((col('MANUFACTURER_CODE').isNull()==True) & (col('NATIONAL_CODE')=="-1")),lit('UNKNOWN PROD')).otherwise(col('MANUFACTURER_CODE')))
                   .withColumn("MANUFACTURER_CODE", when(((col('MANUFACTURER_CODE').isNull()==True) & (col('NATIONAL_CODE')=="-9")),lit('NA-FREE CODE')).otherwise(col('MANUFACTURER_CODE')))
                   .withColumn("MANUFACTURER_CODE", when(((col('MANUFACTURER_CODE').isNull()==True) & (col('NATIONAL_CODE')>"0")),lit('-1')).otherwise(col('MANUFACTURER_CODE')))
                ).alias('df_detail') 

    ADP_log_debug(process, logger_name, level_action, log_level, "[ Assigned -9, -1 default values]  ", sys._getframe().f_code.co_name)
	
    # Unpersist Dataframes
    df_product2.unpersist()
    df_product.unpersist()
    df_product_cat.unpersist()
    df_pharmacy.unpersist()
    df_manufacturers_mdm.unpersist()
    df_product_mdm.unpersist()
    
    ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name)  
    return df_detail
  
  except Exception as err:
      ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
      raise Exception(err)




#############################################################################################################################

def EnrichmentValidationsFarmaticSellout(df,debug=__DEBUG_DEFAULT__,partitions=__PARTITIONS_DEFAULT__):
  """Validate Sellout Enriched Columns Values and Add a Column for each field with the validation Result

    Parameters:
      df                   -- Dataframe with the result of the Enrichment Process
      debug                -- True for enable debug verbosing or False to not enable
      partitions           -- Partitions defined

    Return:
      Dataframe            -- Dataframe containing output columns from Enrichment and the new enrichment validation columns
  """
  #Who                 When           What
  #Victor Salesa       18/12/2018     Create Function Stub
  #Ana Perez           24/12/2018     Validations Enrichment Process
  #Ana Perez           24/04/2019     Included new QA files (NUM_ERROR_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE)
  
  try:
    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)
    df_detail= df.alias('df_detail')
    
    ################################################################################################################################################
    #  BEGIN QA ENRICHMENT DATA FIELDS
    ################################################################################################################################################

    ADP_log_debug(process, logger_name, level_action, log_level, "--Calculate EnrichResult fields", sys._getframe().f_code.co_name)
    
    #Assign EnrichNationalCodeResult, EnrichCategoryResult, EnrichManufacturerResult AND EnrichErrorRow
    df_detail = (df_detail
                    #Validate NATIONAL_CODE
                   .withColumn("EnrichNationalCodeResult",
                           when((col('NATIONAL_CODE')==lit("-9")) | (col('NATIONAL_CODE')!=lit("-1")), lit("OK")))
                   .withColumn("EnrichNationalCodeResult",
                           when(
                             (col('NATIONAL_CODE')==lit("-1")) & 
                             (col('ProductCode')> "599999"),lit("NF1")).otherwise(col('EnrichNationalCodeResult')))
                   .withColumn("EnrichNationalCodeResult",
                           when(
                             (col('NATIONAL_CODE')==lit("-1")) & 
                             (col('ProductCode')> "149999") &
                             (col('ProductCode')< "600000")
                             ,lit("NF2")).otherwise(col('EnrichNationalCodeResult')))
                    # Validate NATIONAL_CODE
                   .withColumn("NATIONAL_CODE_ERR",  when(((col('EnrichNationalCodeResult')=="OK") | (col('NATIONAL_CODE')==lit("-9"))), lit(None)) \
                                .otherwise(concat(lit('{"ENR_UNK:"'), col('EnrichNationalCodeResult') ,lit('"}') )))
                   .withColumn("NUM_ERROR_FIELDS_ACCURACY", when(((col('EnrichNationalCodeResult')=="OK") | (col('NATIONAL_CODE')==lit("-9"))),col("NUM_ERROR_FIELDS_ACCURACY")).\
                                                                 otherwise(col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())))

                    #Validate PRODUCT_NAME
                   .withColumn("PRODUCT_NAME_ERR",  when(((col('PRODUCT_NAME').isNull()==False) | (col('NATIONAL_CODE')<lit("0"))), lit(None)).otherwise('{"' + 'ENR_EMPTY' + '":' + '"1"}') )
                   .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when(((col('PRODUCT_NAME').isNull()==False) | (col('NATIONAL_CODE')<lit("0"))),col("NUM_ERROR_FIELDS_COMPLETENESS")).\
                                                                 otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())))
                    # Validate CLASS
                   .withColumn("CLASS_ERR",  when(((col('CLASS')!=lit('UNKNOWN')) | (col('NATIONAL_CODE')<lit("0"))), lit(None)).otherwise('{"' + 'ENR_EMPTY' + '":' + '"1"}') ) 
                   .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when(((col('CLASS')!=lit('UNKNOWN')) | (col('NATIONAL_CODE')<lit("0"))),col("NUM_ERROR_FIELDS_COMPLETENESS")).\
                                                                 otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())))

                    #Validate CATEGORY
                   .withColumn("EnrichCategoryResult",
                             when((col('CATEGORY')!=lit('UNKNOWN')), lit("OK")))
                   .withColumn("EnrichCategoryResult",
                             when(
                               ((col('CATEGORY')==lit('UNKNOWN'))) & 
                               (col('ProductCode')> "599999")
                               ,lit("NF1")).otherwise(col('EnrichCategoryResult')))
                    .withColumn("EnrichCategoryResult",
                             when(
                               ((col('CATEGORY')==lit('UNKNOWN'))) & 
                               (col('ProductCode')> "149999") &
                               (col('ProductCode')< "600000")
                               ,lit("NF2")).otherwise(col('EnrichCategoryResult')))
                    .withColumn("EnrichCategoryResult",
                             when((col('ProductCode')< "150000")
                               ,lit("NF3")).otherwise(col('EnrichCategoryResult')))
                     # Validate CATEGORY
                    .withColumn("CATEGORY_ERR",  when(((col('EnrichCategoryResult')=="OK") | (col('NATIONAL_CODE')==lit("-9"))), lit(None)).otherwise(concat(lit('{"ENR_EMPTY:"'), col('EnrichCategoryResult') ,lit('"}') )))
                    .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when(((col('EnrichCategoryResult')=="OK") | (col('NATIONAL_CODE')==lit("-9"))),col("NUM_ERROR_FIELDS_COMPLETENESS")).\
                                                                 otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())))
                     # Validate MANUFACTURER
                    .withColumn("MANUFACTURER_NAME_ERR",  when(((col('MANUFACTURER_NAME'))!=lit('UNKNOWN')) | (col('NATIONAL_CODE')<lit("0")), lit(None)).otherwise('{"' + 'ENR_EMPTY' + '":' + '"1"}') )
                    .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when(((col('MANUFACTURER_NAME'))!=lit('UNKNOWN')) | (col('NATIONAL_CODE')<lit("0")),col("NUM_ERROR_FIELDS_COMPLETENESS")).\
                                                                 otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())))
                 
                    .withColumn("MANUFACTURER_CODE_ERR",  when(((col('MANUFACTURER_CODE')!="-1") | (col('NATIONAL_CODE')<lit("0"))), lit(None)).otherwise('{"' + 'ENR_EMPTY' + '":' + '"1"}') )
                    .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when(((col('MANUFACTURER_CODE')!="-1") | (col('NATIONAL_CODE')<lit("0"))),col("NUM_ERROR_FIELDS_COMPLETENESS")).\
                                                                 otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())))
                 
                     # Validate BRAND
                    .withColumn("BRAND_ERR",  when(((col('BRAND')!=lit('UNKNOWN')) | (col('NATIONAL_CODE')<lit("0"))), lit(None)).otherwise('{"' + 'ENR_EMPTY' + '":' + '"1"}') )  
                    .withColumn("NUM_ERROR_FIELDS_COMPLETENESS", when(((col('BRAND')!=lit('UNKNOWN')) | (col('NATIONAL_CODE')<lit("0"))),col("NUM_ERROR_FIELDS_COMPLETENESS")).\
                                                                 otherwise(col("NUM_ERROR_FIELDS_COMPLETENESS")+lit(1).cast(IntegerType())))
                 
                     # Validate CUSTOMER_CODE(PHARMACY_CODE from pharmacies)
                    .withColumn("CUSTOMER_CODE_ERR",  when((col('CUSTOMER_CODE')!="-1"), lit(None)).otherwise('{"' + 'ENR_UNK' + '":' + '"1"}') )
                    .withColumn("NUM_ERROR_FIELDS_ACCURACY", when((col('CUSTOMER_CODE')!="-1"),col("NUM_ERROR_FIELDS_ACCURACY")).\
                                                                 otherwise(col("NUM_ERROR_FIELDS_ACCURACY")+lit(1).cast(IntegerType())))
                     # Validate NUMBER_VALIDATED_FIELDS
                     .withColumn("ROW_OK"
                               , when((col('NUM_ERROR_FIELDS_COMPLETENESS') +
                                 col('NUM_ERROR_FIELDS_ACCURACY') +
                                 col('NUM_ERROR_FIELDS_DUPLICATE') +
                                 col('NUM_ERROR_FIELDS_CONFORMITY')
                                     )>0, lit(1)
                                     ).otherwise(lit(0))
                               )
                 ).alias('df_detail') 
    
    #end if enrichErrorRow
    ################################################################################################################################################
    #  END - QA ENRICHMENT DATA FIELDS
    ################################################################################################################################################
    ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name)
    return df_detail
  
  except Exception as err:
    ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
    raise Exception(err)

#############################################################################################################################

def GenerateTemporaryFarmaticSellout(df,debug=__DEBUG_DEFAULT__,partitions=__PARTITIONS_DEFAULT__):
  """From the EnrichmentValidations Process output, generate a Dataframe with Sellout Canonical final structure to save to a temporary File just in case
      the process fails and we need to recovery the information
      
  Parameters:
    df                   -- Dataframe with the result of the EnrichmentValidation Process
    debug                -- True for enable debug verbosing or False to not enable
    partitions           -- Partitions defined

  Return:
    Dataframe            -- Sell-Out canonized data and QA fields as _RAW and _ERR fields 
  """
  #Who                 When           What
  #Ana Perez           22/02/2018     Create Function Stub
  #Ana Perez           15/04/2019     Included debug log managment and exception
  #Victor Salesa       29/04/2019     Included new QA fields (NUM_VALIDATED_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE)
  #Victor Salesa       29/04/2019     Included new QA fields (NUM_ERROR_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE)
  #Victor Salesa       29/04/2019     Renamed JOB_ID field with NUM_ROWS_TOTAL
  try:

    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)
    df_detail= df.alias('df_detail')
    
	
    ################################################################################################################################################
    #  RENAME FIELDS FOR CANONICAL DATA FILE
    ################################################################################################################################################
    num_files_tot = df_detail.count()
    ADP_log_debug(process, logger_name, level_action, log_level, "Files Selected: " + str(num_files_tot), sys._getframe().f_code.co_name)
	
    df_CMD = (df_detail
                  .select(
                        lit(num_files_tot).alias('NUM_ROWS_TOTAL')
                        ,col('FILE_SPEC_VERSION')
                        ,col('FILE_RELEASE_VERSION')
                        ,col('FILE_ORIGIN')
                        ,col('PMS_CODE')
                        ,col('FILE_TYPE')               
                        ,col('FILE_NAME')
                        ,col('RAW_NAME')
                        ,col('FileDate').alias("FILE_DATE")
                        ,col('LANDING_DATE')
                        ,col('PROCESS_DATE')         
                        ,col('FILE_LINE_NUM') 
                        ,col('PharmacyID').alias('PHARMACY_CODE') 
                        ,col('ExternalPharmacyID').alias('EXTERNAL_PHARMACY_CODE')
                        ,col('CUSTOMER_CODE')
                        ,col('ZIPCode').alias('POSTAL_CODE')
                        ,col('COUNTRY').alias('COUNTRY_CODE')               
                        ,col('OperationIdent').alias('OPERATION_TYPE')
                        ,col('OperationID').alias('OPERATION_CODE')
                        ,col('OperationLine').alias('OPERATION_LINE')
                        ,col('OperationDate').alias('OPERATION_DATE')
                        ,col('POSIdent').alias('SALES_POINT_CODE')
                        ,col('SalesChannel').alias('SALES_CHANNEL')
                        ,col('ProductCode').alias('PRODUCT_LINE_CODE')
                        ,col('ProductName').alias('PRODUCT_LINE_NAME') 
                        ,col('EANCode').alias('PRODUCT_LINE_BAR_CODE') 
                        ,col('NATIONAL_CODE').alias('PRODUCT_NATIONAL_CODE')                
                        ,col('PRODUCT_NAME')        
                        ,lit('').alias('PRODUCT_INTERNAL_CODE') # It will be an Global MDM product code in the future
                        ,col('BRAND')
                        ,col('MANUFACTURER_CODE') 
                        ,col('MANUFACTURER_NAME')
                        ,col('CLASS').alias('LEGAL_CATEGORY')
                        ,col('CATEGORY').alias('COMMERCIAL_CATEGORY_L1')
                        ,col('FAMILY').alias('COMMERCIAL_CATEGORY_L2')
                        ,col('SUBFAMILY').alias('COMMERCIAL_CATEGORY_L3')
                        ,col('ProductPacks').alias('PRODUCT_QTY')
                        ,lit(0).cast(DecimalType(10,0)).alias('PRODUCT_FREE_QTY') # Is not used by Farmatic
                        ,col('ProductPackSize').alias('PACK_SIZE')   
                        ,col('ProductPriceCatalog').alias('PRODUCT_PRICE_CATALOG') 
                        ,col('ProductPrice').alias('PRODUCT_PRICE') 
                        ,col('DiscountType').alias('DISCOUNT_TYPE') 
                        ,col('DiscountValue').alias('DISCOUNT_VALUE') 
                        ,col('DISCOUNT_WEIGTHED_AMOUNT').alias('DISCOUNT_WEIGTHED_AMOUNT') 
                        ,col('ProductFee').alias('PRODUCT_FEE_VALUE') 
                        ,col('ProductMarkup').alias('PRODUCT_MARKUP_VALUE')  
                        ,col('PRODUCT_NET_PRICE').alias('PRODUCT_NET_PRICE') 
                        ,col('PRODUCT_NET_WEIGTHED_PRICE').alias('PRODUCT_NET_WEIGTHED_PRICE') 
                        ,col('ProductTotalPrice').alias('PRODUCT_TOTAL_AMOUNT')  
                        ,col('CoPaymentValue').alias('CONSUMER_PAYMENT_AMOUNT') 
                        ,col('ReimbursementValue').alias('REIMBURSEMENT_AMOUNT') 
                        ,col('ConsumerValue').alias('CONSUMER_AMOUNT') 
                        ,col('DiscountValOvertTotReceipt').alias('TOT_RECEIPT_DISCOUNT_AMOUNT') 
                        ,col('DecreasedValTotReceipt').alias('TOT_RECEIPT_DECREASE_AMOUNT') 
                        ,col('TotalOfReceipt').alias('TOT_RECEIPT_AMOUNT') 
                        ,col('PaymentMode').alias('PAYMENT_MODE') 
                        ,col('ConsumerType').alias('CONSUMER_TYPE') 
                        ,col('ConsumerAnonymousID').alias('CONSUMER_ANONYMOUS_CODE') 
                        ,col('ConsumerGender').alias('CONSUMER_GENDER') 
                        ,col('ConsumerAge').alias('CONSUMER_BIRTH_YEAR') 
                        ,col('ConsumerLocation').alias('CONSUMER_LOCATION_CODE') 
                        ,col('ConsumerZipCode').alias('CONSUMER_POSTAL_CODE')                  
                        ,col('PrescriptionIdent').alias('PRESCRIPTION_FLG') 
                        ,col('PrescriptionMode').alias('PRESCRIPTION_MODE') 
                        ,col('PrescriptionType').alias('PRESCRIPTION_TYPE') 
                        ,col('PrescriptionID').alias('PRESCRIPTION_CODE') 
                        ,col('PrescriptionOrigin').alias('PRESCRIPTION_ORIGIN') 
                        ,col('PrescriptionSpecialtyDoc').alias('PRESCRIPTION_SPECIALITY') 
                        ,col('PrescribedProductCode').alias('PRESCRIPTION_PRODUCT_CODE') 
                        ,col('PrescribedProductName').alias('PRESCRIPTION_PRODUCT_NAME') 
                        ,col('Reimbursement').alias('REIMBURSEMENT_FLG') 
                        ,col('ReimbursementEntityCode').alias('REIMBURSEMENT_ENTITY_CODE') 
                        ,col('ReimbursementEntityName').alias('REIMBURSEMENT_ENTITY_NAME') 
                        ,col('ReimbursementType').alias('REIMBURSEMENT_TYPE') 
                        ,col('ReimbursementCategory').alias('REIMBURSEMENT_CATEGORY') 
                        ,col('Promotion').alias('PROMOTION_FLG') 
                        ,col('PromotionType').alias('PROMOTION_TYPE') 
                        ,col('PromotionID').alias('PROMOTION_CODE') 
                        ,col('PromotionDescription').alias('PROMOTION_DESC') 
                        ,col('IssuedVouchersIdent').alias('ISSUED_VOUCHERS_FLG') 
                        ,col('NumberIssuedVouchers').alias('ISSUED_VOUCHERS_NUM') 
                        ,col('ValueIssuedVouchers').alias('ISSUED_VOUCHERS_AMOUNT') 
                        ,col('UsedVouchersIdent').alias('USED_VOUCHERS_FLG') 
                        ,col('NumberUsedVouchers').alias('USED_VOUCHERS_NUM') 
                        ,col('ValueUsedVouchers').alias('USED_VOUCHERS_AMOUNT') 
                        ,col('LoyaltyProgramUsed').alias('LOYALTY_PROGRAM_FLG') 
                        ,col('LoyaltyProgramName').alias('LOYALTY_PROGRAM_NAME') 
                        ,col('LoyaltyRebate').alias('LOYALTY_REBATE_FLG') 
                        ,lit("SL").alias("BUSINESS_AREA")
                        ,col('FILE_LINES').cast(IntegerType()).alias('FILE_LINES')
                        ,col('OperationLine_Err2').alias('OPERATION_LINE_ERR') 
                        ,col('OperationDate_Err2').alias('OPERATION_DATE_ERR') 
                        ,col('TotalOfReceipt_Err2').alias('TOT_RECEIPT_AMOUNT_ERR') 
                        ,col('DiscountValOvertTotReceipt_Err2').alias('TOT_RECEIPT_DISCOUNT_AMOUNT_ERR') 
                        ,col('DecreasedValTotReceipt_Err2').alias('TOT_RECEIPT_DECREASE_AMOUNT_ERR') 
                        ,col('ProductCode_Err2').alias('PRODUCT_LINE_CODE_ERR') 
                        ,col('ProductPacks_Err2').alias('PRODUCT_QTY_ERR') 
                        ,col('ProductPackSize_Err2').alias('PACK_SIZE_ERR') 
                        ,col('ProductPriceCatalog_Err2').alias('PRODUCT_PRICE_CATALOG_ERR') 
                        ,col('ProductPrice_Err2').alias('PRODUCT_PRICE_ERR') 
                        ,col('DiscountValue_Err2').alias('DISCOUNT_VALUE_ERR') 
                        ,col('ProductTotalPrice_Err2').alias('PRODUCT_TOTAL_AMOUNT_ERR') 
                        ,col('ReimbursementValue_Err2').alias('REIMBURSEMENT_AMOUNT_ERR') 
                        ,col('ConsumerValue_Err2').alias('CONSUMER_AMOUNT_ERR') 
                        ,col('PaymentMode_Err2').alias('PAYMENT_MODE_ERR') 
                        ,col('ConsumerGender_Err2').alias('CONSUMER_GENDER_ERR') 
                        ,col('OperationIdent_Err2').alias('OPERATION_TYPE_ERR') 
                        ,col('PrescriptionIdent_Err2').alias('PRESCRIPTION_FLG_ERR')  
                        ,col('PrescriptionMode_Err2').alias('PRESCRIPTION_MODE_ERR')  
                        ,col('PrescriptionType_Err2').alias('PRESCRIPTION_TYPE_ERR') 
                        ,col('Reimbursement_Err2').alias('REIMBURSEMENT_FLG_ERR') 
                        ,col('NATIONAL_CODE_ERR').alias('NATIONAL_CODE_ERR')
                        ,col('PRODUCT_NAME_ERR').alias('PRODUCT_NAME_ERR') 
                        ,col('CLASS_ERR').alias('LEGAL_CATEGORY_ERR')
                        ,col('CATEGORY_ERR').alias('COMMERCIAL_CATEGORY_L1_ERR')
                        ,col('MANUFACTURER_CODE_ERR')
                        ,col('ConsumerType_Err2').alias('CONSUMER_TYPE_ERR')
                        ,col('MANUFACTURER_NAME_ERR').alias('MANUFACTURER_NAME_ERR') 
                        ,col('BRAND_ERR').alias('BRAND_ERR') 
                        ,col('CUSTOMER_CODE_ERR').alias('CUSTOMER_CODE_ERR') 
                        ,col('OPERATION_LINE_RAW')
                        ,col('OPERATION_DATE_RAW')
                        ,col('CUSTOMER_CODE_RAW')
                        ,col('NATIONAL_CODE_RAW')
                        ,col('PRODUCT_LINE_CODE_RAW')
                        ,col('PRODUCT_NAME_RAW')
                        ,col('CLASS_RAW').alias('LEGAL_CATEGORY_RAW')
                        ,col('CATEGORY_RAW').alias('COMMERCIAL_CATEGORY_L1_RAW')
                        ,col('MANUFACTURER_CODE_RAW')
                        ,col('CONSUMER_TYPE_RAW')
                        ,col('MANUFACTURER_NAME_RAW')
                        ,col('BRAND_RAW')
                        ,col('TOT_RECEIPT_AMOUNT_RAW')
                        ,col('TOT_RECEIPT_DISCOUNT_AMOUNT_RAW')
                        ,col('TOT_RECEIPT_DECREASE_AMOUNT_RAW')
                        ,col('PRODUCT_QTY_RAW')
                        ,col('PACK_SIZE_RAW')
                        ,col('PRODUCT_PRICE_CATALOG_RAW')
                        ,col('PRODUCT_PRICE_RAW')
                        ,col('DISCOUNT_VALUE_RAW')
                        ,col('PRODUCT_TOTAL_AMOUNT_RAW')
                        ,col('REIMBURSEMENT_AMOUNT_RAW')
                        ,col('CONSUMER_AMOUNT_RAW')
                        ,col('PAYMENT_MODE_RAW')
                        ,col('CONSUMER_GENDER_RAW')
                        ,col('OPERATION_TYPE_RAW')
                        ,col('PRESCRIPTION_MODE_RAW')
                        ,col('PRESCRIPTION_TYPE_RAW')
                        ,col('PRESCRIPTION_FLG_RAW')
                        ,col('REIMBURSEMENT_FLG_RAW')
                        ,col('ROW_OK')
                        ,col("NUM_VALIDATED_FIELDS_COMPLETENESS")
                        ,col("NUM_VALIDATED_FIELDS_ACCURACY")
                        ,col("NUM_VALIDATED_FIELDS_DUPLICATE")
                        ,col("NUM_VALIDATED_FIELDS_CONFORMITY")
                        ,col("NUM_ERROR_FIELDS_COMPLETENESS")
                        ,col("NUM_ERROR_FIELDS_ACCURACY")
                        ,col("NUM_ERROR_FIELDS_DUPLICATE")
                        ,col("NUM_ERROR_FIELDS_CONFORMITY")
                        )
             ).alias('df_CMD')
    ADP_log_debug(process, logger_name, level_action, log_level, "After select fields for CMD", sys._getframe().f_code.co_name)
    df_CMD = (df_CMD
                .withColumn('FILE_SPEC_VERSION', (col('FILE_SPEC_VERSION').substr(0,__CMD_FILE_SPEC_VERSION__)))
                .withColumn('FILE_RELEASE_VERSION', (col('FILE_RELEASE_VERSION').substr(0,__CMD_FILE_RELEASE_VERSION__)))
                .withColumn('FILE_ORIGIN', (col('FILE_ORIGIN').substr(0,__CMD_FILE_ORIGIN__)))
                .withColumn('PMS_CODE', (col('PMS_CODE').substr(0,__CMD_PMS_CODE__)))
                .withColumn('FILE_TYPE', (col('FILE_TYPE').substr(0,__CMD_FILE_TYPE__)))
                .withColumn('FILE_NAME', (col('FILE_NAME').substr(0,__CMD_FILE_NAME__)))
                .withColumn('PHARMACY_CODE', (col('PHARMACY_CODE').substr(0,__CMD_PHARMACY_CODE__)))
                .withColumn('EXTERNAL_PHARMACY_CODE', (col('EXTERNAL_PHARMACY_CODE').substr(0,__CMD_EXTERNAL_PHARMACY_CODE__)))
                .withColumn('CUSTOMER_CODE', (col('CUSTOMER_CODE').substr(0,__CMD_CUSTOMER_CODE__)))
                .withColumn('POSTAL_CODE', (col('POSTAL_CODE').substr(0,__CMD_POSTAL_CODE__)))
                .withColumn('COUNTRY_CODE', (col('COUNTRY_CODE').substr(0,__CMD_COUNTRY_CODE__)))
                .withColumn('OPERATION_TYPE', (col('OPERATION_TYPE').substr(0,__CMD_OPERATION_TYPE__)))
                .withColumn('OPERATION_CODE', (col('OPERATION_CODE').substr(0,__CMD_OPERATION_CODE__)))
                .withColumn('SALES_POINT_CODE', (col('SALES_POINT_CODE').substr(0,__CMD_SALES_POINT_CODE__)))
                .withColumn('SALES_CHANNEL', (col('SALES_CHANNEL').substr(0,__CMD_SALES_CHANNEL__)))
                .withColumn('PRODUCT_LINE_CODE', (col('PRODUCT_LINE_CODE').substr(0,__CMD_PRODUCT_LINE_CODE__)))
                .withColumn('PRODUCT_LINE_NAME', (col('PRODUCT_LINE_NAME').substr(0,__CMD_PRODUCT_LINE_NAME__)))
                .withColumn('PRODUCT_LINE_BAR_CODE', (col('PRODUCT_LINE_BAR_CODE').substr(0,__CMD_PRODUCT_LINE_BAR_CODE__)))
                .withColumn('PRODUCT_NATIONAL_CODE', (col('PRODUCT_NATIONAL_CODE').substr(0,__CMD_PRODUCT_NATIONAL_CODE__)))
                .withColumn('PRODUCT_NAME', (col('PRODUCT_NAME').substr(0,__CMD_PRODUCT_NAME__)))
                .withColumn('PRODUCT_INTERNAL_CODE', (col('PRODUCT_INTERNAL_CODE').substr(0,__CMD_PRODUCT_INTERNAL_CODE__)))
                .withColumn('BRAND', (col('BRAND').substr(0,__CMD_BRAND__)))
                .withColumn('MANUFACTURER_CODE', (col('MANUFACTURER_CODE').substr(0,__CMD_MANUFACTURER_CODE__)))
                .withColumn('MANUFACTURER_NAME', (col('MANUFACTURER_NAME').substr(0,__CMD_MANUFACTURER_NAME__)))
                .withColumn('LEGAL_CATEGORY', (col('LEGAL_CATEGORY').substr(0,__CMD_LEGAL_CATEGORY__)))
                .withColumn('COMMERCIAL_CATEGORY_L1', (col('COMMERCIAL_CATEGORY_L1').substr(0,__CMD_COMMERCIAL_CATEGORY_L1__)))
                .withColumn('COMMERCIAL_CATEGORY_L2', (col('COMMERCIAL_CATEGORY_L2').substr(0,__CMD_COMMERCIAL_CATEGORY_L2__)))
                .withColumn('COMMERCIAL_CATEGORY_L3', (col('COMMERCIAL_CATEGORY_L3').substr(0,__CMD_COMMERCIAL_CATEGORY_L3__)))
                .withColumn('DISCOUNT_TYPE', (col('DISCOUNT_TYPE').substr(0,__CMD_DISCOUNT_TYPE__)))
                .withColumn('PAYMENT_MODE', (col('PAYMENT_MODE').substr(0,__CMD_PAYMENT_MODE__)))
                .withColumn('CONSUMER_TYPE', (col('CONSUMER_TYPE').substr(0,__CMD_CONSUMER_TYPE__)))
                .withColumn('CONSUMER_ANONYMOUS_CODE', (col('CONSUMER_ANONYMOUS_CODE').substr(0,__CMD_CONSUMER_ANONYMOUS_CODE__)))
                .withColumn('CONSUMER_GENDER', (col('CONSUMER_GENDER').substr(0,__CMD_CONSUMER_GENDER__)))
                .withColumn('CONSUMER_LOCATION_CODE', (col('CONSUMER_LOCATION_CODE').substr(0,__CMD_CONSUMER_LOCATION_CODE__)))
                .withColumn('CONSUMER_POSTAL_CODE', (col('CONSUMER_POSTAL_CODE').substr(0,__CMD_CONSUMER_POSTAL_CODE__)))
                .withColumn('PRESCRIPTION_FLG', (col('PRESCRIPTION_FLG').substr(0,__CMD_PRESCRIPTION_FLG__)))
                .withColumn('PRESCRIPTION_MODE', (col('PRESCRIPTION_MODE').substr(0,__CMD_PRESCRIPTION_MODE__)))
                .withColumn('PRESCRIPTION_TYPE', (col('PRESCRIPTION_TYPE').substr(0,__CMD_PRESCRIPTION_TYPE__)))
                .withColumn('PRESCRIPTION_CODE', (col('PRESCRIPTION_CODE').substr(0,__CMD_PRESCRIPTION_CODE__)))
                .withColumn('PRESCRIPTION_ORIGIN', (col('PRESCRIPTION_ORIGIN').substr(0,__CMD_PRESCRIPTION_ORIGIN__)))
                .withColumn('PRESCRIPTION_SPECIALITY', (col('PRESCRIPTION_SPECIALITY').substr(0,__CMD_PRESCRIPTION_SPECIALITY__)))
                .withColumn('PRESCRIPTION_PRODUCT_CODE', (col('PRESCRIPTION_PRODUCT_CODE').substr(0,__CMD_PRESCRIPTION_PRODUCT_CODE__)))
                .withColumn('PRESCRIPTION_PRODUCT_NAME', (col('PRESCRIPTION_PRODUCT_NAME').substr(0,__CMD_PRESCRIPTION_PRODUCT_NAME__)))
                .withColumn('REIMBURSEMENT_FLG', (col('REIMBURSEMENT_FLG').substr(0,__CMD_REIMBURSEMENT_FLG__)))
                .withColumn('REIMBURSEMENT_ENTITY_CODE', (col('REIMBURSEMENT_ENTITY_CODE').substr(0,__CMD_REIMBURSEMENT_ENTITY_CODE__)))
                .withColumn('REIMBURSEMENT_ENTITY_NAME', (col('REIMBURSEMENT_ENTITY_NAME').substr(0,__CMD_REIMBURSEMENT_ENTITY_NAME__)))
                .withColumn('REIMBURSEMENT_TYPE', (col('REIMBURSEMENT_TYPE').substr(0,__CMD_REIMBURSEMENT_TYPE__)))
                .withColumn('REIMBURSEMENT_CATEGORY', (col('REIMBURSEMENT_CATEGORY').substr(0,__CMD_REIMBURSEMENT_CATEGORY__)))
                .withColumn('PROMOTION_FLG', (col('PROMOTION_FLG').substr(0,__CMD_PROMOTION_FLG__)))
                .withColumn('PROMOTION_TYPE', (col('PROMOTION_TYPE').substr(0,__CMD_PROMOTION_TYPE__)))
                .withColumn('PROMOTION_CODE', (col('PROMOTION_CODE').substr(0,__CMD_PROMOTION_CODE__)))
                .withColumn('PROMOTION_DESC', (col('PROMOTION_DESC').substr(0,__CMD_PROMOTION_DESC__)))
                .withColumn('ISSUED_VOUCHERS_FLG', (col('ISSUED_VOUCHERS_FLG').substr(0,__CMD_ISSUED_VOUCHERS_FLG__)))
                .withColumn('USED_VOUCHERS_FLG', (col('USED_VOUCHERS_FLG').substr(0,__CMD_USED_VOUCHERS_FLG__)))
                .withColumn('LOYALTY_PROGRAM_FLG', (col('LOYALTY_PROGRAM_FLG').substr(0,__CMD_LOYALTY_PROGRAM_FLG__)))
                .withColumn('LOYALTY_PROGRAM_NAME', (col('LOYALTY_PROGRAM_NAME').substr(0,__CMD_LOYALTY_PROGRAM_NAME__)))
                .withColumn('LOYALTY_REBATE_FLG', (col('LOYALTY_REBATE_FLG').substr(0,__CMD_LOYALTY_REBATE_FLG__)))
                .withColumn('LOAD_DATE', to_timestamp(current_timestamp(), "yyyyMMddHHmmss")) 
             )
    ADP_log_debug(process, logger_name, level_action, log_level, "After cut fields", sys._getframe().f_code.co_name)
    df_CMD = df_CMD.persist(StorageLevel.MEMORY_AND_DISK).alias('df_CMD')
    df_CMD.count()
    
    # ###############################################################################################################################################
    #  END - RENAME FIELDS FOR CANONICAL DATA FILE
    # ###############################################################################################################################################

    ################################################################################################################################################
    #  SAVE TEMPORARY DATA FILE
    ################################################################################################################################################
	#Save the datraframe as a file
    ADP_log_debug(process, logger_name, level_action, log_level, "Before saveAsCanonical TMP_STG_T_SELL_OUT: " + str(df_CMD.count()) + " rows ", sys._getframe().f_code.co_name)
    saveAsCanonical(df_CMD,__PHARMATIC_CANONICAL_STG_SELLOUT_TMP_FILE_PATH__,table_name=__PHARMATIC_CANONICAL_STG_SELLOUT_TMP_H_TABLE_NAME__,mode='overwrite',debug=debug)
    ADP_log_debug(process, logger_name, level_action, log_level, "After saveAsCanonical TMP_STG_T_SELL_OUT", sys._getframe().f_code.co_name)
	
      ##############################################################################################################################################
    #  END - SAVE TEMPORARY DATA FILE 
    ##############################################################################################################################################
    
    ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name)
    return df_CMD
  
  except Exception as err:
    ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
    raise Exception(err)
    
##############################################################################################################################################
def RecoverTemporaryFarmaticSellout(debug=__DEBUG_DEFAULT__,partitions=__PARTITIONS_DEFAULT__):
  """Recover output datafrom
      
  Parameters:
    debug                -- True for enable debug verbosing or False to not enable
    partitions           -- Partitions defined

  Return:
    Dataframe            -- Sell-Out canonized data and QA fields as _RAW and _ERR fields 
  """
  #Who                 When           What
  #Victor Salesa       04/04/2019     Initial Version
  #Ana Perez           15/04/2019     Included debug log managment and exception
  
  try:

    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)
    
    tmp_stg_t_sell_out_df = spark.table(__PHARMATIC_CANONICAL_STG_SELLOUT_TMP_H_TABLE_NAME__)
    
    if callSafe(tmp_stg_t_sell_out_df,"count",0) != 0:
        return tmp_stg_t_sell_out_df
    else:
        ADP_log_exception(process, logger_name, level_action, log_level,  "Table TMP_STG_T_SELL_OUT is empty or Not Found", sys._getframe().f_code.co_name,  sys.exc_info())
        raise Exception('Table TMP_STG_T_SELL_OUT is empty or Not Found')
    #end if callSafe(tmp_stg_t_sell_out_df,"count",0) != 0

    ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name)
  
  except Exception as err:
    ADP_log_exception(process, logger_name, level_action, log_level,  "END Fail reading TMP_STG_T_SELL_OUT", sys._getframe().f_code.co_name,  sys.exc_info())
    raise Exception(err)


def GenerateQAFarmaticSellout(df,debug=__DEBUG_DEFAULT__,partitions=__PARTITIONS_DEFAULT__,job_id=''):
    """From the EnrichmentValidations Process output generate Dataframes with QA final structure to save to DDBB and File

      Parameters:
        df                   -- Dataframe with the result of the EnrichmentValidation Process
        debug                -- True for enable debug verbosing or False to not enable
        partitions           -- Partitions defined
        
      Return:
        Boolean              -- Process Succed or Not
    """
    #Who                 When           What
    #Victor Salesa       18/12/2018     Create Function Stub
    #Victor Salesa       18/01/2019      
    #Victor Salesa       18/01/2019  
    #Victor Salesa       30/01/2019     Added MANUFACTURER_CODE_ERR,MANUFACTURER_CODE_RAW,CONSUMER_TYPE_ERR,CONSUMER_TYPE_RAW
    #                                   Renamed CLASS TO LEGAL_CATEGORY _RAW & _ERR and CATEGORY to COMMERCIAL_CATEGORY_L1 _RAW & _ERR         
    #                                   Added "PMS_CODE","COUNTRY_CODE","PHARMACY_CODE","BUSINESS_AREA" to output
    #Victor Salesa       30/01/2019     Change "ProcessFileErrorDF = ProcessFileErrorDF.distinct()" before write file
    #Ana Pérez           26/02/2019     Adapted to new Temporary source Dataframe 
    #Victor Salesa       17/04/2019     Added job_id logic and Rollback Logic and moved all to STG tables
    #Victor Salesa       29/04/2019     Included new QA fields (NUM_ERROR_FIELDS_COMPLETENESS, CONFORMITY, ACCURACY, DUPLICATE)
    #Victor Salesa       29/04/2019     Renamed JOB_ID field with NUM_ROWS_TOTAL
    #Victor Salesa       29/04/2019     Drop Cols ROW_KO_PMS,ROW_KO_MDM,NUMBER_OF_VALIDATED_FIELDS_PMS,NUMBER_OF_VALIDATED_FIELDS_MDM,"OLD" ROW_OK, ROW_KO
    
    ##### Prepare fields for processfileerror ###########################################################################################################################################################################
    try:
      ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)
      exception_status=''
      ProcessFileQADF= (df
                         .select(
                           col('FILE_NAME')
                          ,col("PMS_CODE")
                          ,col("COUNTRY_CODE")
                          ,col("PHARMACY_CODE")
                          ,col("BUSINESS_AREA")
                          ,col("LANDING_DATE")
                          ,col("PROCESS_DATE")
                          ,col('FILE_LINE_NUM')
                          ,col('FILE_LINES').cast(IntegerType()).alias('FILE_LINES')
                          ,col('OPERATION_LINE_ERR') 
                          ,col('OPERATION_DATE_ERR') 
                          ,col('TOT_RECEIPT_AMOUNT_ERR') 
                          ,col('TOT_RECEIPT_DISCOUNT_AMOUNT_ERR') 
                          ,col('TOT_RECEIPT_DECREASE_AMOUNT_ERR') 
                          ,col('PRODUCT_LINE_CODE_ERR') 
                          ,col('PRODUCT_QTY_ERR') 
                          ,col('PACK_SIZE_ERR') 
                          ,col('PRODUCT_PRICE_CATALOG_ERR') 
                          ,col('PRODUCT_PRICE_ERR') 
                          ,col('DISCOUNT_VALUE_ERR') 
                          ,col('PRODUCT_TOTAL_AMOUNT_ERR') 
                          ,col('REIMBURSEMENT_AMOUNT_ERR') 
                          ,col('CONSUMER_AMOUNT_ERR') 
                          ,col('PAYMENT_MODE_ERR') 
                          ,col('CONSUMER_GENDER_ERR') 
                          ,col('OPERATION_TYPE_ERR') 
                          ,col('PRESCRIPTION_FLG_ERR')  
                          ,col('PRESCRIPTION_MODE_ERR')  
                          ,col('PRESCRIPTION_TYPE_ERR') 
                          ,col('REIMBURSEMENT_FLG_ERR') 
                          ,col('NATIONAL_CODE_ERR')
                          ,col('PRODUCT_NAME_ERR') 
                          ,col('LEGAL_CATEGORY_ERR')
                          ,col('COMMERCIAL_CATEGORY_L1_ERR')
                          ,col('MANUFACTURER_CODE_ERR')
                          ,col('CONSUMER_TYPE_ERR')
                          ,col('MANUFACTURER_NAME_ERR') 
                          ,col('BRAND_ERR') 
                          ,col('CUSTOMER_CODE_ERR') 
                          ,col('OPERATION_LINE_RAW')
                          ,col('OPERATION_DATE_RAW')
                          ,col('CUSTOMER_CODE_RAW')
                          ,col('NATIONAL_CODE_RAW')
                          ,col('PRODUCT_LINE_CODE_RAW')
                          ,col('PRODUCT_NAME_RAW')
                          ,col('LEGAL_CATEGORY_RAW')
                          ,col('COMMERCIAL_CATEGORY_L1_RAW')
                          ,col('MANUFACTURER_CODE_RAW')
                          ,col('CONSUMER_TYPE_RAW')
                          ,col('MANUFACTURER_NAME_RAW')
                          ,col('BRAND_RAW')
                          ,col('TOT_RECEIPT_AMOUNT_RAW')
                          ,col('TOT_RECEIPT_DISCOUNT_AMOUNT_RAW')
                          ,col('TOT_RECEIPT_DECREASE_AMOUNT_RAW')
                          ,col('PRODUCT_QTY_RAW')
                          ,col('PACK_SIZE_RAW')
                          ,col('PRODUCT_PRICE_CATALOG_RAW')
                          ,col('PRODUCT_PRICE_RAW')
                          ,col('DISCOUNT_VALUE_RAW')
                          ,col('PRODUCT_TOTAL_AMOUNT_RAW')
                          ,col('REIMBURSEMENT_AMOUNT_RAW')
                          ,col('CONSUMER_AMOUNT_RAW')
                          ,col('PAYMENT_MODE_RAW')
                          ,col('CONSUMER_GENDER_RAW')
                          ,col('OPERATION_TYPE_RAW')
                          ,col('PRESCRIPTION_MODE_RAW')
                          ,col('PRESCRIPTION_TYPE_RAW')
                          ,col('PRESCRIPTION_FLG_RAW')
                          ,col('REIMBURSEMENT_FLG_RAW')
                          ,col('NUM_ROWS_TOTAL')
                          ,col('ROW_OK')
                          ,col("NUM_VALIDATED_FIELDS_COMPLETENESS")
                          ,col("NUM_VALIDATED_FIELDS_ACCURACY")
                          ,col("NUM_VALIDATED_FIELDS_DUPLICATE")
                          ,col("NUM_VALIDATED_FIELDS_CONFORMITY")
                          ,col("NUM_ERROR_FIELDS_COMPLETENESS")
                          ,col("NUM_ERROR_FIELDS_ACCURACY")
                          ,col("NUM_ERROR_FIELDS_DUPLICATE")
                          ,col("NUM_ERROR_FIELDS_CONFORMITY")
                   )                
                  ).persist(StorageLevel.MEMORY_AND_DISK)
      ProcessFileQADF.count()

      ADP_log_debug(process, logger_name, level_action, log_level, "---Prepare fields for process_file_error", sys._getframe().f_code.co_name)
	  
      ##### Drop fields with no errors inside ###########################################################################################################################################################################

      def qa_drop_empty_columns(df):
        """
        This function drops all columns which contain null values.
        :param df: A PySpark DataFrame
        """
        not_null_counts = df.select([count(when(col(c).isNotNull(), c)).alias(c) for c in df.columns if '_ERR' in c]).collect()[0].asDict()
        to_drop_err = [k for k, v in not_null_counts.items() if v == 0 and '_ERR' in k]
        to_drop_raw = [column.replace('_ERR', '_RAW') for column in to_drop_err]
        to_drop = to_drop_err + to_drop_raw
        df = df.drop(*to_drop)
        return df

      ProcessFileErrorDataDF = (qa_drop_empty_columns(ProcessFileQADF)
                                 .drop("NUM_ROWS_TOTAL"
                                      ,"ROW_OK"
                                      ,"NUM_VALIDATED_FIELDS_COMPLETENESS"
                                      ,"NUM_VALIDATED_FIELDS_ACCURACY"
                                      ,"NUM_VALIDATED_FIELDS_DUPLICATE"
                                      ,"NUM_VALIDATED_FIELDS_CONFORMITY"
                                      ,"NUM_ERROR_FIELDS_COMPLETENESS"
                                      ,"NUM_ERROR_FIELDS_ACCURACY"
                                      ,"NUM_ERROR_FIELDS_DUPLICATE"
                                      ,"NUM_ERROR_FIELDS_CONFORMITY"
                                      ,"FILE_LINES"
                                 )
                                 .persist(StorageLevel.MEMORY_AND_DISK)
                             )

      ProcessFileErrorDataDF.count()
      
      ADP_log_debug(process, logger_name, level_action, log_level, "---Drop Unwanted Err fields", sys._getframe().f_code.co_name)
	  
      ##### Generate process_file_error QA data ###########################################################################################################################################################################
      ProcessFileErrorDF= QA_CTL_PROCESS_FILE_ERROR_DATA(ProcessFileErrorDataDF,debug=True).persist(StorageLevel.MEMORY_AND_DISK)
      ProcessFileErrorDF.count()
    
      ADP_log_debug(process, logger_name, level_action, log_level, "---Generate process_file_error QA data", sys._getframe().f_code.co_name)
      
	  ##### Save to file process_file_error QA data #######################################################################################################################################################################
      ProcessFileErrorDF = ProcessFileErrorDF.distinct() 
      
      ADP_log_debug(process, logger_name, level_action, log_level, "---Before Save to file process_file_error QA data " + str(ProcessFileErrorDF.count()) + " rows", sys._getframe().f_code.co_name)
      saveAsCanonical(ProcessFileErrorDF,__QUALITY_PFE_H_FILE_PATH__,table_name=__QUALITY_PFE_H_TABLE_NAME__,mode='append',debug=debug,job_id=job_id)
      ADP_log_debug(process, logger_name, level_action, log_level, "---After Save to file process_file_error QA data", sys._getframe().f_code.co_name)

      exception_status = "GenerateQAFarmaticSellout.CTL_PROCESS_FILE_ERROR_CANONICAL"
      
      ##### Save to DB process_file_error QA data #########################################################################################################################################################################
      ADP_log_debug(process, logger_name, level_action, log_level, "---Before Save to DB process_file_error QA data " + str(ProcessFileErrorDF.count()) + " rows", sys._getframe().f_code.co_name)
      saveToDB(ProcessFileErrorDF,table_name=__QUALITY_PFE_DB_TABLE_NAME__,mode="append",debug=debug,job_id=job_id)  
      ADP_log_debug(process, logger_name, level_action, log_level, "---After Save to DB process_file_error QA data", sys._getframe().f_code.co_name)
	 
      exception_status = "GenerateQAFarmaticSellout.CTL_PROCESS_FILE_ERROR_DB"
      
      #####Generate process_file_qa QA data##############################################################################################################################################################################
      ProcessFileQADF = (ProcessFileQADF
      
        .groupBy("FILE_NAME", "LANDING_DATE", "PROCESS_DATE", "PMS_CODE", "COUNTRY_CODE", "PHARMACY_CODE", "BUSINESS_AREA"
          ,"NUM_ROWS_TOTAL"
          ,"NUM_VALIDATED_FIELDS_COMPLETENESS"
          ,"NUM_VALIDATED_FIELDS_ACCURACY"
          ,"NUM_VALIDATED_FIELDS_DUPLICATE"
          ,"NUM_VALIDATED_FIELDS_CONFORMITY"
        )
        .agg(
          sum(col("ROW_OK")).alias("NUM_ROWS_OK"),
          sum(col("NUM_ERROR_FIELDS_COMPLETENESS")).alias("NUM_ERROR_FIELDS_COMPLETENESS"),
          sum(col("NUM_ERROR_FIELDS_ACCURACY")).alias("NUM_ERROR_FIELDS_ACCURACY"),
          sum(col("NUM_ERROR_FIELDS_DUPLICATE")).alias("NUM_ERROR_FIELDS_DUPLICATE"),
          sum(col("NUM_ERROR_FIELDS_CONFORMITY")).alias("NUM_ERROR_FIELDS_CONFORMITY")

        )
        .withColumn("STATUS", lit(1))
        .distinct()
      ).persist(StorageLevel.MEMORY_AND_DISK)

      ProcessFileQADF.count()

      ADP_log_debug(process, logger_name, level_action, log_level, "---Generate ProcessFileQADF", sys._getframe().f_code.co_name)
      
      ##### Save to file process_file_qa QA data #######################################################################################################################################################################
      ADP_log_debug(process, logger_name, level_action, log_level, "---Before Save to file process_file_qa QA data: " + str(ProcessFileQADF.count()) + " rows", sys._getframe().f_code.co_name)    
      saveAsCanonical(ProcessFileQADF,__QUALITY_PFQ_H_FILE_PATH__,table_name=__QUALITY_PFQ_H_TABLE_NAME__,mode='append',debug=debug,job_id=job_id)
      
      exception_status = "GenerateQAFarmaticSellout.CTL_PROCESS_FILE_QA_CANONICAL"
     
      ADP_log_debug(process, logger_name, level_action, log_level, "---After Save to file process_file_qa QA data", sys._getframe().f_code.co_name)    
        
     ##### Save to DB process_file_qa QA data ########################################################################################################################################################################
      ADP_log_debug(process, logger_name, level_action, log_level, "---Before Save to DB process_file_qa QA data: " + str(ProcessFileQADF.count()) + " rows", sys._getframe().f_code.co_name)   
      saveToDB(ProcessFileQADF,table_name=__QUALITY_PFQ_DB_TABLE_NAME__,mode="append",debug=debug,job_id=job_id)
      ADP_log_debug(process, logger_name, level_action, log_level, "---After Save to DB process_file_qa QA data", sys._getframe().f_code.co_name)    
  
      exception_status = "GenerateQAFarmaticSellout.CTL_PROCESS_FILE_QA_DB"

      ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name)
      return True
	  
    except Exception as err:
      ADP_log_debug(process, logger_name, level_action, log_level, "EXCEPTION_STATUS:"+exception_status, sys._getframe().f_code.co_name)
      if exception_status == 'GenerateQAFarmaticSellout.CTL_PROCESS_FILE_ERROR_DB':
        RollbackCanonicalTable(__QUALITY_PFE_H_TABLE_NAME__,rollback_id=job_id)
      elif exception_status == 'GenerateQAFarmaticSellout.CTL_PROCESS_FILE_QA_CANONICAL':
        RollbackCanonicalTable(__QUALITY_PFE_H_TABLE_NAME__,rollback_id=job_id)
        RollbackDBTable(__QUALITY_PFE_DB_TABLE_NAME__,rollback_id=job_id)
      elif exception_status == 'GenerateQAFarmaticSellout.CTL_PROCESS_FILE_QA_DB':
        RollbackCanonicalTable(__QUALITY_PFE_H_TABLE_NAME__,rollback_id=job_id)
        RollbackDBTable(__QUALITY_PFE_DB_TABLE_NAME__,rollback_id=job_id)
        RollbackCanonicalTable(__QUALITY_PFQ_H_TABLE_NAME__,rollback_id=job_id)
      #end if exception_status == 'CTL_PROCESS_FILE_ERROR_DB':
      
      ADP_log_exception(process, logger_name, level_action, log_level, "", sys._getframe().f_code.co_name,  sys.exc_info())
      raise Exception(err)
    

#############################################################################################################################

def GenerateCanonicalFarmaticSellout(df,debug=__DEBUG_DEFAULT__,partitions=__PARTITIONS_DEFAULT__,job_id=''):
  """From the EnrichmentValidations Process output generate Dataframes with Sellout Canonical final structure to save to DDBB and File

  Parameters:
    df                   -- Dataframe with the result of the EnrichmentValidation Process
    debug                -- True for enable debug verbosing or False to not enable
    partitions           -- Partitions defined

  Return:
    Boolean              -- Process Succed or Not 
  """
  #Who                 When           What
  #Victor Salesa       18/12/2018     Create Function Stub
  #Ana Perez           10/01/2019     Save Canonical File and Database
  #Victor Salesa       16/01/2019     Delete Processed Files 
  #Ana Perez           25/01/2019     Including new calculated fields 
  #Ana Perez           28/01/2019     Extract from string fields the length allowed in the table 
  #Victor Salesa       28/01/2019     Corrected delete function
  #Ana Perez           11/02/2019     Replace delete_file function to blob_delete_file_sql
  #Ana Perez           13/02/2019     Add LOAD_DATE to T_SELL_OUT file and STG_T_SELL_OUT table
  #Ana Perez           25/02/2019     Including new GenerateTemporaryFarmaticSellout-output as source data
  #Ana Perez           11/04/2019     Included log managment and exception managment
  #Ana Perez           15/04/2019     Included Debug log managment
  #Victor Salesa       22/04/2019     Added Job Id parameter and moved files and table names to variables and removed old JOB_id
  try:
    exception_status = "GenerateCanonicalFarmaticSellout.START_PROCESS"
    
    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)
    
    df_detail= df.alias('df_detail')
    
    ################################################################################################################################################
    #  RENAME FIELDS FOR CANONICAL DATA FILE
    ################################################################################################################################################
    
    df_CMD = (df_detail
              .select(
                      col('FILE_SPEC_VERSION')
                     ,col('FILE_RELEASE_VERSION')
                     ,col('FILE_ORIGIN')
                     ,col('PMS_CODE')
                     ,col('FILE_TYPE')               
                     ,col('FILE_NAME')
                     ,col('FILE_DATE')
                     ,col('LANDING_DATE')
                     ,col('PROCESS_DATE')         
                     ,col('FILE_LINE_NUM') 
                     ,col('PHARMACY_CODE')
                     ,col('EXTERNAL_PHARMACY_CODE')
                     ,col('CUSTOMER_CODE')
                     ,col('POSTAL_CODE')
                     ,col('COUNTRY_CODE')             
                     ,col('OPERATION_TYPE')
                     ,col('OPERATION_CODE')
                     ,col('OPERATION_LINE')
                     ,col('OPERATION_DATE')
                     ,col('SALES_POINT_CODE')
                     ,col('SALES_CHANNEL')
                     ,col('PRODUCT_LINE_CODE')
                     ,col('PRODUCT_LINE_NAME') 
                     ,col('PRODUCT_LINE_BAR_CODE') 
                     ,col('PRODUCT_NATIONAL_CODE')                
                     ,col('PRODUCT_NAME')        
                     ,col('PRODUCT_INTERNAL_CODE') 
                     ,col('BRAND')
                     ,col('MANUFACTURER_CODE') 
                     ,col('MANUFACTURER_NAME')
                     ,col('LEGAL_CATEGORY')
                     ,col('COMMERCIAL_CATEGORY_L1')
                     ,col('COMMERCIAL_CATEGORY_L2')
                     ,col('COMMERCIAL_CATEGORY_L3')
                     ,col('PRODUCT_QTY')
                     ,col('PRODUCT_FREE_QTY') 
                     ,col('PACK_SIZE')   
                     ,col('PRODUCT_PRICE_CATALOG') 
                     ,col('PRODUCT_PRICE') 
                     ,col('DISCOUNT_TYPE') 
                     ,col('DISCOUNT_VALUE') 
                     ,col('DISCOUNT_WEIGTHED_AMOUNT') 
                     ,col('PRODUCT_FEE_VALUE') 
                     ,col('PRODUCT_MARKUP_VALUE')  
                     ,col('PRODUCT_NET_PRICE') 
                     ,col('PRODUCT_NET_WEIGTHED_PRICE') 
                     ,col('PRODUCT_TOTAL_AMOUNT')  
                     ,col('CONSUMER_PAYMENT_AMOUNT') 
                     ,col('REIMBURSEMENT_AMOUNT') 
                     ,col('CONSUMER_AMOUNT') 
                     ,col('TOT_RECEIPT_DISCOUNT_AMOUNT') 
                     ,col('TOT_RECEIPT_DECREASE_AMOUNT') 
                     ,col('TOT_RECEIPT_AMOUNT') 
                     ,col('PAYMENT_MODE') 
                     ,col('CONSUMER_TYPE') 
                     ,col('CONSUMER_ANONYMOUS_CODE') 
                     ,col('CONSUMER_GENDER') 
                     ,col('CONSUMER_BIRTH_YEAR') 
                     ,col('CONSUMER_LOCATION_CODE') 
                     ,col('CONSUMER_POSTAL_CODE')                  
                     ,col('PRESCRIPTION_FLG') 
                     ,col('PRESCRIPTION_MODE') 
                     ,col('PRESCRIPTION_TYPE') 
                     ,col('PRESCRIPTION_CODE') 
                     ,col('PRESCRIPTION_ORIGIN') 
                     ,col('PRESCRIPTION_SPECIALITY') 
                     ,col('PRESCRIPTION_PRODUCT_CODE') 
                     ,col('PRESCRIPTION_PRODUCT_NAME') 
                     ,col('REIMBURSEMENT_FLG') 
                     ,col('REIMBURSEMENT_ENTITY_CODE') 
                     ,col('REIMBURSEMENT_ENTITY_NAME') 
                     ,col('REIMBURSEMENT_TYPE') 
                     ,col('REIMBURSEMENT_CATEGORY') 
                     ,col('PROMOTION_FLG') 
                     ,col('PROMOTION_TYPE') 
                     ,col('PROMOTION_CODE') 
                     ,col('PROMOTION_DESC') 
                     ,col('ISSUED_VOUCHERS_FLG') 
                     ,col('ISSUED_VOUCHERS_NUM') 
                     ,col('ISSUED_VOUCHERS_AMOUNT') 
                     ,col('USED_VOUCHERS_FLG') 
                     ,col('USED_VOUCHERS_NUM') 
                     ,col('USED_VOUCHERS_AMOUNT') 
                     ,col('LOYALTY_PROGRAM_FLG') 
                     ,col('LOYALTY_PROGRAM_NAME') 
                     ,col('LOYALTY_REBATE_FLG') 
                     )
              .withColumn('LOAD_DATE', to_timestamp(current_timestamp(), "yyyyMMddHHmmss")) 
             ).persist(StorageLevel.MEMORY_AND_DISK).alias('df_CMD')
    df_CMD.count()
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Data Selected", sys._getframe().f_code.co_name)
    # ###############################################################################################################################################
    #  END - RENAME FIELDS FOR CANONICAL DATA FILE
    # ###############################################################################################################################################

    ################################################################################################################################################
    #  SAVE CANONICAL DATA FILE
    ################################################################################################################################################
    
    #Save the datraframe as a file
    ADP_log_debug(process, logger_name, level_action, log_level, "Before saveAsCanonical STG_T_SELL_OUT: " + str(df_CMD.count()) + " rows ", sys._getframe().f_code.co_name)
    saveAsCanonical(df_CMD,__PHARMATIC_CANONICAL_STG_SELLOUT_FILE_PATH__,table_name=__PHARMATIC_CANONICAL_STG_SELLOUT_H_TABLE_NAME__,mode='append',debug=True,job_id=job_id)
    ADP_log_debug(process, logger_name, level_action, log_level, "After saveAsCanonical STG_T_SELL_OUT", sys._getframe().f_code.co_name)
	
    exception_status = "GenerateCanonicalFarmaticSellout.STG_T_SELLOUT_CANONICAL"
    
    #refresh LOAD_DATE to Database 
    df_CMD = df_CMD.withColumn('LOAD_DATE', to_timestamp(current_timestamp(), "yyyyMMddHHmmss")) 
    ADP_log_debug(process, logger_name, level_action, log_level, "Before saveToDB STG_T_SELL_OUT: " + str(df_CMD.count()) + " rows ", sys._getframe().f_code.co_name)	
    saveToDB(df_CMD,__PHARMATIC_CANONICAL_STG_SELLOUT_DB_TABLE_NAME__,mode="append",debug=True,job_id=job_id)
    ADP_log_debug(process, logger_name, level_action, log_level, "After saveToDB STG_T_SELL_OUT", sys._getframe().f_code.co_name)
    
    exception_status = "GenerateCanonicalFarmaticSellout.STG_T_SELLOUT_DB"
	
    ##############################################################################################################################################
    #  END - SAVE CANONICAL DATA FILE AND DATABASE
    ##############################################################################################################################################
    
    if __DELETE_FILES_ENABLED__ == True:
      #Delete processed files
      ADP_log_debug(process, logger_name, level_action, log_level, "Before delete processed files", sys._getframe().f_code.co_name)	
      delete_selected = df_detail.select(col('RAW_NAME')).distinct().withColumn("deleted",blob_delete_file_sql(concat(lit(__PHARMATIC_TOBEPROCESSED_BASE_PATH__),col("RAW_NAME"))))
      deleted         = delete_selected.collect()
      ADP_log_debug(process, logger_name, level_action, log_level, "After delete processed files", sys._getframe().f_code.co_name)

    ################################################################################################################################################
    #  END DELETE FILES
    ################################################################################################################################################
    ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name)
    
    return True
  
  except Exception as err:
    if exception_status == "GenerateCanonicalFarmaticSellout.STG_T_SELLOUT_CANONICAL":
      RollbackCanonicalTable(__PHARMATIC_CANONICAL_STG_SELLOUT_H_TABLE_NAME__,rollback_id=job_id)
    #end if exception_status == "GenerateCanonicalFarmaticSellout.STG_T_SELLOUT_CANONICAL"
    
    ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
    raise Exception(err)
    